# aq_pp command examples

In this notebook, we'll go over common usage examples of the data preprocessing command, `aq_pp`. We'll start from very basic, and work our way up to advanced examples.

Before going over this notebook, make sure you're faimilar with 

* Bash commands
* Regular Expression
* aq_input / input-spec 

We won't go over input, column and output spec on this notebook. They can be found on 
- [this notebook](aq_input.ipynb).
- [aq_output notebook](aq_output.ipynb)
 

Also have the [aq_pp documentation](http://auriq.com/documentation/source/reference/manpages/aq_pp.html) ready on your side, so you can refer to the details of each options as you go over this sample.

## ToS

## Basic Options

### Evaluation (Better wording?)

- `-eval`
- `-var`

### Output Options
- `-c`
- `-o`
- `-ovar`

### Combining Datasets
- `-cat`
- `-cmb`
- `-sub`

### Filtering
- `-grep`
- `-filt`


### String Manipulation
- `-mapf ... -mapc`
- `-map`


### Conditionals
- `-if`
- `-elif`
- `-else`
- `-endif`


### Variable usage
### Advanced Options
- `-kenc`
- `-kdec`
- `-pmod`
- `-imp`
- `-exp`(this belongs to input spec, so should be in [aq_input notebook](aq_input.ipynb)?
### Buildin Variables
- `Random`
- `RowNum`
- `CurSec`
- `CurUSec`


## Basic Options
we'll start with basic options for data wrangling. Let's take a look at them by objectives.

### Evalutation

This section explores 2 options, 
- `-eval`: evaluates give expression, and store the result in exsiting or new column.
- `-var`: create a new variable and initialize it's value. 

These 2 options are essential for manipulating and performing calculations on it. 

#### [-eval](http://auriq.com/documentation/source/reference/manpages/aq_pp.html#eval)

Basic syntax for this option is
```bash
aq-pp ... -eval ColSpec|ColName Expr
```
where
- `ColSpec`: new column's column spec to assign the result
- `ColName`: existing column name to assign the result
- `Expr`: expression to be evaluated.

#### Data

[Ramen Ratings Dataset](https://www.kaggle.com/residentmario/ramen-ratings) from kaggle will be used in this sample, which contains ratings of 2500 ramen products. 

Review|Brand|Variety|Style|Country|Stars
---|---|---|---|---|---|
2580|New Touch|T's Restaurant Tantanmen|Cup|Japan|3.75
2579|Just Way|Noodles Spicy Hot Sesame Spicy Hot Sesame Guan-miao Noodles|Pack|Taiwan|1
2578|Nissin|Cup Noodles Chicken Vegetable|Cup|USA|2.25
2577|Wei Lih|GGE Ramen Snack Tomato Flavor|Pack|Taiwan|2.75
2576|Ching's Secret|Singapore Curry|Pack|India|3.75
2575|Samyang Foods|Kimchi song Song Ramen|Pack|South Korea|4.75
2574|Acecook|Spice Deli Tantan Men With Cilantro|Cup|Japan|4
2573|Ikeda Shoku|Nabeyaki Kitsune Udon|Tray|Japan|3.75
2572|Ripe'n'Dry|Hokkaido Soy Sauce Ramen|Pack|Japan|0.25
2571|KOKA|The Original Spicy Stir-Fried Noodles|Pack|Singapore|2.5

Here are its' columns and data types.
- `int: Review #`: review id number, the more recent the review is, the bigger the number is
- `str: Brand`: brand / manufacture of the product
- `str: Variety`: title of the product
- `str: Style`: categorical styles of the products, cup, pack or tray
- `str: Country`: country of origin
- `float: stars`: star rating of each product

And here is the corresponding column specs<br>
`i:reviewID s:brand s:variety s:style s:country f:stars`

**Note**<br>
When reading in the files with `aq-pp`, we'll be using bash's [variable substitution](http://www.compciv.org/topics/bash/variables-and-substitution/).

#### Todo

Operations that can be performed on data by leveraging `-eval` option can be broken down and simplied to 4 categories below.
Things to cover in this section, are
- ONIT: numerical evaluation
    - constant and colname, on new column
    - constant and colName, on existing column
    - colName and ColName, on new column
    - 
- string manipulation
- data type conversion operation
    - data type
- any operation possible by builtin functions (aq-emod)


Let's start with Numerical operations.

**Numerical Operation**

Operators supported for numerical operation are<br>

_Arithmetic_
- `*`: multiplication
- `/`: division
- `%`: modulus
- `+`: addition
- `-`: subtraction

_Bitwise_
- `&`: AND
- `|`: OR
- `^`: XOR

First, we will double the value of star rating column, and assign it to a new column named `double_rating`. 

In [1]:
# First store filename and column spec in variable to simplify commands
file="data/aq_pp/ramen-ratings.csv"
cols="i:reviewID s:brand s:variety s:style s:country f:stars"
# now create a column called double_rating, and assign the value of 2 * stars
aq_pp -f,+1 $file -d $cols -eval f:double_rating '2*stars'

"reviewID","brand","variety","style","country","stars","double_rating"
2580,"New Touch","T's Restaurant Tantanmen ","Cup","Japan",3.75,7.5
2579,"Just Way","Noodles Spicy Hot Sesame Spicy Hot Sesame Guan-miao Noodles","Pack","Taiwan",1,2
2578,"Nissin","Cup Noodles Chicken Vegetable","Cup","USA",2.25,4.5
2577,"Wei Lih","GGE Ramen Snack Tomato Flavor","Pack","Taiwan",2.75,5.5
2576,"Ching's Secret","Singapore Curry","Pack","India",3.75,7.5
2575,"Samyang Foods","Kimchi song Song Ramen","Pack","South Korea",4.75,9.5
2574,"Acecook","Spice Deli Tantan Men With Cilantro","Cup","Japan",4,8
2573,"Ikeda Shoku","Nabeyaki Kitsune Udon","Tray","Japan",3.75,7.5
2572,"Ripe'n'Dry","Hokkaido Soy Sauce Ramen","Pack","Japan",0.25,0.5
2571,"KOKA","The Original Spicy Stir-Fried Noodles","Pack","Singapore",2.5,5
2570,"Tao Kae Noi","Creamy tom Yum Kung Flavour","Pack","Thailand",5,10
2569,"Yamachan","Yokohama Tonkotsu Shoyu","Pack","USA",5,10
2568,"Nongshim","Mr. Bibim Stir-Fried Kimchi Flavor","Pack","South K

2465,"Nissin","Cup Noodles XO Sauce Seafood","Cup","China",3.75,7.5
2464,"Wang","Extreme Spicy Hot Chicken Flavor Udon","Bowl","South Korea",2,4
2463,"Nissin","Gekikara Ramen Rasa Pedas","Pack","Indonesia",3.25,6.5
2462,"TTL","Sesame Oil Chicken Noodle With Rice Wine","Bowl","Taiwan",5,10
2461,"Fujiwara","Hokkaido Hakodate Shio Ramen","Pack","Japan",4.5,9
2460,"Daifuku","Katsuo Bowl Udon","Bowl","USA",5,10
2459,"Nissin","Cup Noodles MUG Shoyu & Seafood","Pack","Japan",3.5,7
2457,"KOKA","Instant Noodles Chicken Satay Flavour","Pack","Singapore",3.5,7
2456,"KOKA","Signature Spicy Singapore Fried Noodles Instant Noodles","Pack","Singapore",4.25,8.5
2455,"KOKA","Creamy Soup With Crushed Noodles Sweet Corn Flavor","Cup","Singapore",5,10
2454,"KOKA","Silk Beef Pho Flavor Instant Rice Fettuccine","Bowl","Singapore",3.25,6.5
2453,"KOKA","Delight Spicy Sesame Flavor Instant Non-Fried Noodles","Pack","Singapore",4.5,9
2452,"KOKA","Signature Laksa Singapura Instant Noodles","Bowl","Singapore",5,1

2360,"Wai Wai","Instant Noodles Artificial Chicken Flavoured","Pack","India",2,4
2359,"Daikoku","Kitsune Udon","Cup","Japan",2,4
2358,"Sichuan Baijia","Pickled Cabbage Flavor Instant Vermicelli","Bowl","China",4,8
2357,"Kamfen","Noodle King Artificial Wonton Soup Flavored","Bowl","Hong Kong",5,10
2356,"Nongshim","Spicy Shrimp Cup Noodle","Cup","South Korea",4,8
2355,"Marutai","Nagasaki Agodashi Shoyu Ramen","Pack","Japan",4,8
2354,"Vifon","Viet Cuisine Bun Rieu Cua Sour Crab Soup Instant Rice Vermicelli","Bowl","Vietnam",5,10
2353,"Nissin","Cup Noodles Rich Garlic, Egg Yolk & Oxtail","Cup","Japan",4,8
2352,"Uni-President","Minced Pork Flavor Instant Noodles","Pack","Taiwan",1,2
2351,"1 To 3 Noodles","Chatpat Masala","Pack","India",4,8
2350,"Nissin","Cup Noodles BIG XO Sauce Seafood Flavour","Cup","Hong Kong",4,8
2349,"Fantastic","Noodles Chicken & Corn Flavour","Cup","Australia",3,6
2348,"Ripe'n'Dry","Hokkaido Salt Ramen","Pack","Japan",3,6
2347,"Nissin","Demae Iccho Tonkotsu Flavour I

2255,"Daikoku","Hiroshima Flavor Yakisoba","Tray","Japan",5,10
2254,"Nissin","Disney Cuties Instant Noodle Seaweed Flavour","Cup","Thailand",3,6
2253,"Singa-Me","Instant Noodles Mi Goreng Flavour","Cup","Australia",4,8
2252,"New Touch","Sugomen Sano Ramen","Bowl","Japan",5,10
2251,"Ottogi","Ramyon Sabor Y Camaron","Pack","Mexico",4,8
2250,"Nissin","Demae Iccho Spicy XO Sauce Seafood Flavour","Pack","Hong Kong",4,8
2249,"Love Cook","Sun Dried Noodle - Fruity Soy Bean Paste","Pack","Taiwan",4,8
2248,"Nongshim","Shin Noodle Soup","Cup","USA",5,10
2247,"Nissin","Sapporo Noukou Miso Ramen","Bowl","Japan",5,10
2246,"Dream Kitchen","Soy Sauce","Cup","USA",4,8
2245,"Nissin","Vietnamese Chicken Coriander Flavour Vermicelli","Cup","Hong Kong",4,8
2244,"Kang Shi Fu","Artificial Spicy Beef Flavour","Pack","Hong Kong",4,8
2243,"Nissin","Cup Noodles Black Pepper Crab Flavour","Cup","Hong Kong",4,8
2242,"Great Value","Beef Style Noodles With Vegetables","Cup","Canada",3,6
2241,"Fuku","Spicy Soup Inst

2146,"Sapporo Ichiban","Kun Ramen Sumo-kuchikin (Smoked Chicken)","Bowl","Japan",3,6
2145,"Dream Kitchen","Tonkotsu","Cup","USA",3,6
2144,"Nissin","Lamen Sabor Picanha","Pack","Brazil",4,8
2143,"Thien Houng Foods","Lemon Chicken Flavour Instant Noodles","Pack","Vietnam",3,6
2142,"Maruchan","Bowl Taste Of Asia Miso Chicken Flavor Spicy Miso Ramen","Bowl","USA",4,8
2141,"Nissin","Men Shokunin Kaoruyasai Shio","Bowl","Japan",5,10
2140,"Wang","Rice Noodle Seafood Flavour","Bowl","South Korea",5,10
2139,"Fuku","Superior Soup Instant Noodles (Mfg in Thailand)","Pack","Hong Kong",4,8
2138,"Fuku","Superior Soup Instant Noodles (Mfg in Vietnam)","Pack","Hong Kong",4,8
2137,"Nissin","Demae Ramen Spicy Curry Flavour Instant Noodles","Pack","Hong Kong",4,8
2136,"Nissin","Demae Ramen Roast Beef Flavour Instant Noodles","Pack","Hong Kong",4,8
2135,"Doll","Instant Noodle Supreme Abalone And Chicken Flavour","Pack","Hong Kong",2,4
2134,"Four Seas","Chicken Flavour Instant Noodle","Pack","Hong Kong",4,

...

568,"Paldo","Gomtang","Cup","South Korea",4,8
567,"Goku Uma","Soy Sauce","Bowl","Japan",3.5,7
566,"Foodmon","DJ DOC Jjolmyeon","Bowl","South Korea",5,10
565,"Batchelors","Super Noodles To Go Curry","Bowl","UK",4,8
564,"Uni-President","One More Cup Kimchi","Cup","Taiwan",3.75,7.5
563,"Yum Yum","Duck","Pack","Thailand",3.25,6.5
562,"Batchelors","Super Noodles Bacon","Pack","UK",4.25,8.5
561,"Sahmyook","Vegetable","Pack","South Korea",3.25,6.5
560,"Myojo","Ippeichan Soy Sauce","Bowl","Japan",4,8
559,"Wai Wai","Quick Tom Yum Shrimp","Cup","Thailand",3,6
558,"Long Kow","Crystal Noodles Vegetables","Bowl","China",1.5,3
557,"Batchelors","Super Noodles Souther Fried Chicken","Pack","UK",3.75,7.5
556,"Mi Sedaap","Mi Goreng Perisa Asli","Pack","Indonesia",4.75,9.5
555,"Batchelors","Super Noodles Chinese Chow Mein","Pack","UK",2.75,5.5
554,"Yum Yum","Shrimp Tom Yum Flavor","Pack","Thailand",3.5,7
553,"Samyang","60 Ingredients","Cup","South Korea",2.75,5.5
552,"Super Bihun","Kuah Rasa Baso Sapi Ri

434,"Mi Sedaap","Kari Spesial","Pack","Indonesia",4.5,9
433,"Lucky Me!","Supreme Bulalo","Bowl","Philippines",3.25,6.5
432,"Ve Wong","Instant Bean Thread Seafood","Bowl","Taiwan",2.5,5
431,"Unif / Tung-I","Chinese Onion Rice Noodle","Pack","Taiwan",2.75,5.5
430,"Quickchow","Pancit Canton toyo Mansi","Pack","Philippines",5,10
429,"Quickchow","Chicken Mami","Pack","Philippines",3.75,7.5
428,"Kamfen","E Menm Chicken",,"China",3.75,7.5
427,"Koka","Mi Hai Cua Crab Flavor","Pack","Singapore",5,10
426,"Vifon","Tu quy Chicken","Pack","Vietnam",3,6
425,"Chikara","Shrimp Udon","Pack","USA",4.5,9
424,"Q","Noodle With Sesame Sauce","Pack","Taiwan",3.25,6.5
423,"Noodle Time","Shin Cup","Cup","South Korea",3,6
422,"Vifon","Mi Lau Thai Thai Stle","Bowl","Vietnam",4,8
421,"Myojo","Kakesoba Desse Soy Sauce","Bowl","Japan",2.75,5.5
420,"Sapporo Ichiban","Beef Flavorc","Cup","USA",3.5,7
419,"Myojo","Ramen Desse Shio","Bowl","Japan",4.25,8.5
418,"iNoodle","Udon Noodle Soup Oriental","Pack","Taiwan",3.75,7

306,"Rhee Bros Assi","Rice Noodle Anchovy Flavor","Tray","South Korea",3.75,7.5
305,"Lucky Me!","Supreme Special Beef Carne De Vaca","Bowl","Philippines",3.75,7.5
304,"Mama","Pho Bo Rice Noodle Artificial Beef","Bowl","Thailand",3.5,7
303,"Paldo","Dosirac Artificial Chicken","Tray","South Korea",3.25,6.5
302,"Samyang","Bowl Noodle Picante Beef","Bowl","South Korea",3.75,7.5
301,"Dragonfly","Guilin Style Rice Vermicelli","Bowl","China",4.5,9
300,"Wu Mu","Steam Spinach Ramen With Onion","Pack","Taiwan",5,10
299,"Lucky Me!","Supreme Sotanghon Artificial Chicken Vermicelli ","Bowl","Philippines",3.75,7.5
298,"Nongshim","Hearty Rice Noodle Spicy Veggie Consomme","Pack","South Korea",2.75,5.5
297,"Rhee Bros Assi","Rice Noodle Hot & Spicy Kimchi","Tray","South Korea",3.25,6.5
296,"Nissin","Cup Noodles Crab","Cup","Hong Kong",3.25,6.5
295,"Tablemark","Soybean Paste","Bowl","Japan",4.5,9
294,"Paldo","South Korean Seafood","Pack","South Korea",3.75,7.5
293,"Tiger","Onion Flaver Rice Noodle","Bow

176,"Paldo","Dosirac Pork","Tray","South Korea",4.125,8.25
175,"Samyang","Cold Ramen With Ponytail Radish","Pack","South Korea",3,6
174,"Lucky Me!","Sotanghon Artificial Chicken","Pack","Philippines",0.25,0.5
173,"Unif / Tung-I","Instant Bean Vermicelli Shrimp","Pack","Taiwan",1.5,3
172,"Mama","Flat Noodle Tom Yum","Pack","Thailand",3.75,7.5
171,"Samyang","Japanese Seafood Flavor Udon","Pack","South Korea",3.75,7.5
170,"Myojo","Chukazanmai Soybean Paste","Pack","Japan",3.5,7
169,"Samyang","Seafood Party","Pack","South Korea",2.25,4.5
168,"JFC","Japanese Style Noodle Fresh Udon","Bowl","Japan",3.75,7.5
167,"Wai Wai","Crab Instant Rice Vermicelli","Pack","Thailand",2.5,5
166,"Koyo","Garlic Pepper","Pack","USA",3.25,6.5
165,"Doll","Sesame Oil","Pack","Hong Kong",3.5,7
164,"Unif / Tung-I","Artificial Beef Flavor","Pack","Taiwan",3.75,7.5
163,"Vifon","Asian Style Instant Noodles Chicken","Pack","Vietnam",3.5,7
162,"Myojo","Chukazanmai Oriental","Pack","Japan",3.75,7.5
161,"Paldo","Green Tea

50,"Indomie","Mi Goreng","Pack","Indonesia",5,10
49,"Indomie","Mi Goreng Pedas Hot","Pack","Indonesia",4.5,9
48,"Indomie","Shrimp Flavor","Pack","Indonesia",3,6
47,"Indomie","Mi Goreng Jumbo Barbecue Chicken","Pack","Indonesia",5,10
46,"Indomie","Mi Goreng Jumbo Meatball","Pack","Indonesia",3,6
45,"Indomie","Mi Goreng Sate","Pack","Indonesia",5,10
44,"Indomie","Special Chicken","Pack","Indonesia",4.25,8.5
43,"Kim Ve Wong","Jaopai Series: Vegetarian Instant Noodles","Bowl","Taiwan",0,0
42,"Ve Wong","Kung-Fu Chicken Flavor","Pack","Vietnam",2.75,5.5
41,"Little Cook","Pork Mustard Stem","Bowl","Thailand",0.5,1
40,"Lucky Me!","Pancit Canton Chili Mansi","Pack","Philippines",3,6
39,"Lucky Me!","Supreme Bulalo Flavor","Bowl","Philippines",3,6
38,"Lucky Me!","Pancit Canton Citrus Flavor","Pack","Philippines",3,6
37,"Lucky Me!","Pancit Canton Hot Chili Flavor","Pack","Philippines",3,6
36,"Lucky Me!","Itnok","Pack","Philippines",3,6
35,"Mama","Chand Clear Soup","Pack","Thailand",3,6
34,"Marucha

Now the new column `double_rating` contains the value twice as large as the `stars` value.

**Couple things to note**<br>
- **Column Datatype:** the destination column's datatype has to be same as the datatype of result of the `Expr`. In the example above, the result is float datatype, therefore we've declared `double_rating` as float.
- **Quotations:** you cannot quote `colName|colSpec`, while `Expr` to be evaluated needs to be quoted. Single quotation is recommended, because string value in the expression needs to be quoted as well.

Now we will perform the same operation, but store the result on existing column, `stars`.

In [2]:
aq_pp -f,+1 $file -d $cols -eval stars '2*stars'

"reviewID","brand","variety","style","country","stars"
2580,"New Touch","T's Restaurant Tantanmen ","Cup","Japan",7.5
2579,"Just Way","Noodles Spicy Hot Sesame Spicy Hot Sesame Guan-miao Noodles","Pack","Taiwan",2
2578,"Nissin","Cup Noodles Chicken Vegetable","Cup","USA",4.5
2577,"Wei Lih","GGE Ramen Snack Tomato Flavor","Pack","Taiwan",5.5
2576,"Ching's Secret","Singapore Curry","Pack","India",7.5
2575,"Samyang Foods","Kimchi song Song Ramen","Pack","South Korea",9.5
2574,"Acecook","Spice Deli Tantan Men With Cilantro","Cup","Japan",8
2573,"Ikeda Shoku","Nabeyaki Kitsune Udon","Tray","Japan",7.5
2572,"Ripe'n'Dry","Hokkaido Soy Sauce Ramen","Pack","Japan",0.5
2571,"KOKA","The Original Spicy Stir-Fried Noodles","Pack","Singapore",5
2570,"Tao Kae Noi","Creamy tom Yum Kung Flavour","Pack","Thailand",10
2569,"Yamachan","Yokohama Tonkotsu Shoyu","Pack","USA",10
2568,"Nongshim","Mr. Bibim Stir-Fried Kimchi Flavor","Pack","South Korea",8.5
2567,"Nissin","Deka Buto Kimchi Pork Flavor","Bowl","

2457,"KOKA","Instant Noodles Chicken Satay Flavour","Pack","Singapore",7
2456,"KOKA","Signature Spicy Singapore Fried Noodles Instant Noodles","Pack","Singapore",8.5
2455,"KOKA","Creamy Soup With Crushed Noodles Sweet Corn Flavor","Cup","Singapore",10
2454,"KOKA","Silk Beef Pho Flavor Instant Rice Fettuccine","Bowl","Singapore",6.5
2453,"KOKA","Delight Spicy Sesame Flavor Instant Non-Fried Noodles","Pack","Singapore",9
2452,"KOKA","Signature Laksa Singapura Instant Noodles","Bowl","Singapore",10
2451,"KOKA","Delight Spicy Black Pepper Flavor Instant Non-Fried Noodles","Pack","Singapore",10
2450,"KOKA","Purple Wheat Noodles Chili & Lime Flavor","Pack","Singapore",8
2449,"KOKA","Delight Tomato Flavor Instant Non-Fried Noodles","Pack","Singapore",10
2448,"KOKA","Seafood Flavor Instant Noodles","Cup","Singapore",7
2447,"KOKA","Signature Stir-Fry Original Flavor Instant Noodles","Pack","Singapore",6
2446,"KOKA","Creamy Soup Witrh Crushed Noodles Curry Flavor","Cup","Singapore",10
2445,"KOKA

2349,"Fantastic","Noodles Chicken & Corn Flavour","Cup","Australia",6
2348,"Ripe'n'Dry","Hokkaido Salt Ramen","Pack","Japan",6
2347,"Nissin","Demae Iccho Tonkotsu Flavour Instant Noodle (Bowl Noodle)","Bowl","Hong Kong",6
2346,"Knorr","Chatt Patta Instant Noodles","Pack","Pakistan",2
2345,"Sempio","Clam Flavor Soup Kal-guksu","Pack","South Korea",10
2344,"Nissin","Cup Noodles Sopa Nissin Sabor A Pollo","Cup","Mexico",8
2343,"Nissin","Maxi Sopa Nissin Cuchareable Sabor A Jugo De Carne","Cup","Mexico",8
2342,"Nissin","Sopa Nissin Sabor A Camaron Sopa Instantanea Tipo Ramen","Pack","Mexico",8
2341,"Nissin","Cup Noodles Sopa Nissin Sabor A Carne De Res","Cup","Mexico",8
2340,"Nissin","RapiFideo Sabor A Tomate Y Pollo","Cup","Mexico",8
2339,"Nissin","Cup Noodles Sopa Nissin Sabor Camaron, Habanero Y Limon","Cup","Mexico",6
2338,"Nissin","Sopa Nissin Sabor A Pollo Sopa Instantanea Tipo Ramen","Pack","Mexico",8
2337,"Nissin","Cup Noodles Sopa Nissin Con Camaron","Cup","Mexico",6
2336,"Nissin"

2239,"Nissin","Cup Noodles Chicken Flavor Ramen Noodle Soup (New Recipe)","Cup","USA",8
2238,"Asian Thai Foods","Rumpum Gundruk Flavour","Pack","Nepal",2
2237,"Nissin","Cup Noodle Light+ Bagna Cauda","Cup","Japan",10
2236,"Maruchan","Gotsumori Koku Tonkotsu Ramen","Bowl","Japan",8
2235,"Fashion Food","Tom Yum Shrimp Creamy Flavour Instant Noodles","Bowl","Thailand",8
2234,"Paldo","Bibim Men","Bowl","South Korea",10
2233,"Nissin","Cup Noodles Milk Seafood Flavour","Cup","Hong Kong",8
2232,"Nissin","Spicy Chikin Donburi","Bowl","Japan",10
2231,"Fantastic","Noodle Chicken Flavour","Cup","Australia",8
2230,"Marutai","Kuroma-Yu Tonkotsu Kumamoto Ramen","Pack","Japan",8
2229,"Kang Shi Fu","Artificial Beef With Sauerkraut Flavour","Pack","Hong Kong",8
2228,"Nissin","Raoh Shoyu Ramen","Bowl","Japan",10
2227,"Maruchan","Ramen Noodle Soup Sriracha Chicken Flavor","Pack","USA",8
2226,"Nagatanien","Hiyashi Soba Taidashi Goma","Pack","Japan",6
2225,"Fuku","Bowl Chicken Flavour Instant Rice Vermicel

2128,"Sapporo Ichiban","Taimeiken Yousyoku Yasangatsukutta Omumen","Cup","Japan",8
2127,"Suimin","Noodles With Chicken & Sweet Corn Flavour","Cup","Australia",4.5
2126,"Nissin","Mennippon Oumi Chanpon","Bowl","Japan",10
2125,"Happy Family","Curry Flavour Instant Noodles","Pack","Singapore",8
2124,"7 Select","Nissin Instant Noodles Tom Yum Seafood Creamy Flavour King Cup","Bowl","Thailand",7
2123,"Nissin","Cup Noodles Tom Yum Shrimp Sabb Flavour","Cup","Thailand",10
2122,"Nissin","Chilli Noodles Stir Fried Holy Basil Flavour","Pack","Thailand",10
2121,"Nissin","Instant Noodles Moo Manao Flavour","Pack","Thailand",8
2120,"Nissin","Super Tom Yum Shrimp","Cup","Thailand",7
2119,"Nissin","Cup Noodles Minced Pork Flavour","Cup","Thailand",7
2118,"Nissin","Chilli Noodles Tom Yum Shrimp Creamy Soup Flavour","Cup","Thailand",6.5
2117,"7 Select","Nissin Instant Noodles Shrimp Ma Nao Lui Suan Flavour King Cup","Bowl","Thailand",8
2116,"Nissin","Instant Noodles Tom Yum Shrimp Saab Flavour","Pack",

...

584,"Baijia","Single Noble Black Bone Chicken Sweet Potato Thread","Cup","China",0
583,"Vifon","Phu Gia Cua Crab Bean Thread","Pack","Vietnam",4.5
582,"Koyo","Tofu & Miso","Pack","USA",5
581,"Koyo","Mushroom","Pack","USA",5.5
580,"Baijia","Hot & Sour Casserole Stewed Rice Noodle","Pack","China",0
579,"Baijia","Sichuan Pickled Sweet Potato Thread","Pack","China",3.5
578,"Nongshim","Shin Ramyun Black","Pack","South Korea",9.5
577,"Pot Noodle","Chinese Chow Mein","Cup","UK",6
576,"Menraku","Tempura Soba","Bowl","Japan",7.5
575,"Batchelors","Super Noodles Barbecue Beef","Pack","UK",7
574,"Maggi","2 Minute Noodles Tricky Tomato","Pack","India",7.5
573,"Noah Foods","Phoya Chicken","Bowl","Vietnam",4
572,"Maruchan","Yakisoba Tomato Basil","Tray","USA",6
571,"Shirakiku","Japanese Style Tokusen Sio","Pack","Japan",6.5
570,"O Sung","Keopnurungji Rice Snack","Cup","South Korea",5
569,"Saigon Ve Wong","Kung Fu Suc Song Artificial Pork","Bowl","Vietnam",6
568,"Paldo","Gomtang","Cup","South Korea",8

441,"Ottogi","Spice Seafood","Pack","South Korea",7.5
440,"Maruchan","Instant Lunch Hot spicy Beef","Cup","USA",5.5
439,"Little Cook","Tom Yum Seafood Creamy","Bowl","Thailand",6
438,"Samyang","Beef","Bowl","South Korea",7.5
437,"Nongshim","Big Bowl Spicy Kimchi","Bowl","South Korea",6.5
436,"Unif","Artificial Beef With Sauerkraut","Bowl","Taiwan",7.5
435,"Mi Sedaap","Mie Sambal Goreng","Pack","Indonesia",8.5
434,"Mi Sedaap","Kari Spesial","Pack","Indonesia",9
433,"Lucky Me!","Supreme Bulalo","Bowl","Philippines",6.5
432,"Ve Wong","Instant Bean Thread Seafood","Bowl","Taiwan",5
431,"Unif / Tung-I","Chinese Onion Rice Noodle","Pack","Taiwan",5.5
430,"Quickchow","Pancit Canton toyo Mansi","Pack","Philippines",10
429,"Quickchow","Chicken Mami","Pack","Philippines",7.5
428,"Kamfen","E Menm Chicken",,"China",7.5
427,"Koka","Mi Hai Cua Crab Flavor","Pack","Singapore",10
426,"Vifon","Tu quy Chicken","Pack","Vietnam",6
425,"Chikara","Shrimp Udon","Pack","USA",9
424,"Q","Noodle With Sesame Sauc

304,"Mama","Pho Bo Rice Noodle Artificial Beef","Bowl","Thailand",7
303,"Paldo","Dosirac Artificial Chicken","Tray","South Korea",6.5
302,"Samyang","Bowl Noodle Picante Beef","Bowl","South Korea",7.5
301,"Dragonfly","Guilin Style Rice Vermicelli","Bowl","China",9
300,"Wu Mu","Steam Spinach Ramen With Onion","Pack","Taiwan",10
299,"Lucky Me!","Supreme Sotanghon Artificial Chicken Vermicelli ","Bowl","Philippines",7.5
298,"Nongshim","Hearty Rice Noodle Spicy Veggie Consomme","Pack","South Korea",5.5
297,"Rhee Bros Assi","Rice Noodle Hot & Spicy Kimchi","Tray","South Korea",6.5
296,"Nissin","Cup Noodles Crab","Cup","Hong Kong",6.5
295,"Tablemark","Soybean Paste","Bowl","Japan",9
294,"Paldo","South Korean Seafood","Pack","South Korea",7.5
293,"Tiger","Onion Flaver Rice Noodle","Bowl","Taiwan",0
292,"Paldo","South Korean U Dong","Pack","South Korea",7.5
291,"Nongshim","Shin Bowl","Bowl","South Korea",6
290,"Doll","Spicy Artificial Beef","Pack","Hong Kong",6
289,"Paldo","South Korean Broad N

166,"Koyo","Garlic Pepper","Pack","USA",6.5
165,"Doll","Sesame Oil","Pack","Hong Kong",7
164,"Unif / Tung-I","Artificial Beef Flavor","Pack","Taiwan",7.5
163,"Vifon","Asian Style Instant Noodles Chicken","Pack","Vietnam",7
162,"Myojo","Chukazanmai Oriental","Pack","Japan",7.5
161,"Paldo","Green Tea Chlorella","Pack","South Korea",0
160,"Myojo","Ramen Desse Soy Sauce","Bowl","Japan",3
159,"Mama","Hot & Spicy","Pack","Thailand",3.5
158,"Mama","Jok Cup Porridge Artificial Pork","Cup","Thailand",5
157,"Nissin","Sotanghon","Cup","Philippines",4
156,"Maruchan","Instant Lunch Cajun Style With chili Piquin Shrimp","Cup","USA",6
155,"Lucky Me!","Pancit Canton","Pack","Philippines",9.5
154,"Nissin","Creamy Chicken","Cup","USA",3.5
153,"Sapporo Ichiban","Shrimp Flavor","Cup","USA",3.5
152,"Wu Mu","Steamed Noodle Garlic & Sesame Oil","Pack","Taiwan",9
151,"Shirakiku","Japanese Style Tokusen Miso","Pack","Japan",6
150,"Chorip Dong","Jjambbong Hot Spicy Seafood","Bowl","South Korea",8.5
149,"Marucha

27,"Nissin","Demae Ramen Curry Flavor","Pack","Japan",8
26,"Nissin","Soba Noodles With Mayo Mustard Squirt","Pack","Japan",8
25,"Nissin","Demae Ramen Spicy Flavor","Pack","Japan",6
24,"Nissin","Demae Ramen Spicy Seafood With Chili Pepper","Pack","Japan",4
23,"Nissin","Top Ramen Creamy Chicken","Pack","USA",9
22,"Nongshim","Ansungtangmyun Noodle Soup","Pack","South Korea",7.5
21,"Nongshim","Champong Oriental Noodles","Pack","South Korea",8
20,"Nongshim","Neoguri (Seafood'n'Spicy)","Pack","South Korea",7
19,"Nongshim","Shin Ramyun","Pack","South Korea",8
18,"Ottogi","Jin Ramen (Hot Taste)","Pack","South Korea",7
17,"Quickchow","Pancit Palabok","Pack","Philippines",5
16,"Samyang","Kalgug-Su (Spicy)","Pack","South Korea",7
15,"Samyang","Pojangmacha U-dong","Pack","South Korea",5
14,"Samyang","Hot","Pack","South Korea",7
13,"Sapporo Ichiban","Chow Mein","Pack","Japan",10
12,"Sapporo Ichiban","Shrimp Flavor","Pack","Japan",5
11,"Six Fortune","Chicken Flavor Instant Soup Noodle","Pack","South

You can apply any of the other arithmetic operators just like above example. 

In the above example, `Expr` only contained existing column and a constant. We can also provide multiple columns as `Expr` and perform calculation.

We'll divide the `reviewID` (int) by `stars`(float), and store the result in new column `div`(float).

In [3]:
aq_pp -f,+1 $file -d $cols -eval f:div 'reviewID/stars'

"reviewID","brand","variety","style","country","stars","div"
2580,"New Touch","T's Restaurant Tantanmen ","Cup","Japan",3.75,688
2579,"Just Way","Noodles Spicy Hot Sesame Spicy Hot Sesame Guan-miao Noodles","Pack","Taiwan",1,2579
2578,"Nissin","Cup Noodles Chicken Vegetable","Cup","USA",2.25,1145.7777777777778
2577,"Wei Lih","GGE Ramen Snack Tomato Flavor","Pack","Taiwan",2.75,937.09090909090912
2576,"Ching's Secret","Singapore Curry","Pack","India",3.75,686.93333333333328
2575,"Samyang Foods","Kimchi song Song Ramen","Pack","South Korea",4.75,542.10526315789468
2574,"Acecook","Spice Deli Tantan Men With Cilantro","Cup","Japan",4,643.5
2573,"Ikeda Shoku","Nabeyaki Kitsune Udon","Tray","Japan",3.75,686.13333333333333
2572,"Ripe'n'Dry","Hokkaido Soy Sauce Ramen","Pack","Japan",0.25,10288
2571,"KOKA","The Original Spicy Stir-Fried Noodles","Pack","Singapore",2.5,1028.4000000000001
2570,"Tao Kae Noi","Creamy tom Yum Kung Flavour","Pack","Thailand",5,514
2569,"Yamachan","Yokohama Tonkotsu S

2480,"Acecook","Pork Wantan Men","Bowl","Japan",4.25,583.52941176470586
2479,"E-Zee","Instant Noodles Chicken Flavour","Pack","Malaysia",4,619.75
2478,"Kiki Noodle","Scallion Oil & Soy Sauce Noodle","Pack","Taiwan",5,495.60000000000002
2477,"Kiki Noodle","Sichuan Spices Flavor Noodle","Pack","Taiwan",5,495.39999999999998
2476,"Samyang Foods","Kimchi Stew Ramyun","Bowl","South Korea",4,619
2475,"Maruchan","Seimen Red Spicy Dandan Men","Bowl","Japan",4.75,521.0526315789474
2474,"Ottogi","Ppushu Ppushu Noodle Snack Chilli Cheese Flavor","Pack","South Korea",4.25,582.11764705882354
2473,"Tokyo Noodle","Mini Instant Noodle Mild Curry Flavor","Pack","Japan",2,1236.5
2472,"GGE","Noodle Snack Wheat Cracks Seaweed Flavor","Pack","Taiwan",3.5,706.28571428571433
2471,"Nissin","Cup Noodles Mini Seafood Flavour","Cup","Hong Kong",5,494.19999999999999
2470,"Myojo","Udon Japanese Style Noodles With Soup Base Hot & Sour Flavor","Pack","USA",3.75,658.66666666666663
2469,"KOKA","Signature Curry Flavor I

2387,"Nissin","Demae Iccho Sesame Oil Flavour Instant Noodle","Bowl","Hong Kong",3.5,682
2386,"Maggi","2 Minute Noodles Curry Flavour","Pack","Fiji",4,596.5
2385,"Uni-President","Man Han Feast Chilli Beef Flavor Instant Noodles","Bowl","Taiwan",4.75,502.10526315789474
2384,"Sichuan Guangyou","9999 Chongqing Artificial Beef Flavor Instant Noodle","Pack","China",4.25,560.94117647058829
2383,"Nissin","Cup Noodles Beef Flavor Ramen Noodle Soup (New Recipe)","Cup","USA",3.5,680.85714285714289
2382,"Nongshim","Seaweed Instant Noodle","Cup","South Korea",0.5,4764
2381,"Nissin","Demae Iccho Seafood Flavour Instant Noodle","Cup","Hong Kong",4.75,501.26315789473682
2380,"Pulmuone","Non-Fried Ramyun With Spicy Beef Broth","Pack","South Korea",3.75,634.66666666666663
2379,"Nissin","Cup Noodles Hot & Spicy Shrimp Flavor Ramen Noodle Soup (New Recipe)","Cup","USA",2.5,951.60000000000002
2378,"Sau Tao","Non-Fried Mix Noodle Black Pepper XO Sauce Flavoured","Pack","Hong Kong",4.5,528.44444444444446
23

2292,"Sichuan Baijia","Chongqing Noodles Spicy Hot Flavor","Pack","China",5,458.39999999999998
2291,"Samyang Foods","Buldak Bokkummyun Cheese Flavor","Pack","South Korea",4,572.75
2290,"Nissin","Demae Ramen Straight Noodle Black Garlic Oil Tonkotsu Flavour Instant Noodle","Pack","Hong Kong",5,458
2289,"Myojo","Shin Toyama Black Ramen","Bowl","Japan",4,572.25
2288,"Nissin","Cup Noodles Tom Yum Goong Flavour","Cup","Hong Kong",4,572
2287,"Master Kong","Artificial Abalone Crab Flavor Instant Noodle","Pack","China",0,inf
2286,"Kamfen","Noodle King Artificial Abalone And Chicken Soup Flavored","Bowl","Hong Kong",4,571.5
2285,"Fantastic","Noodles Beef Flavour","Cup","Australia",3,761.66666666666663
2284,"Nissin","Korean Kimchi Flavour Vermicelli","Cup","Hong Kong",4,571
2283,"Nissin","Raoh Pork Bone Soy Soup Noodle","Pack","Japan",5,456.60000000000002
2282,"Samyang Foods","Gold Jjamppong Fried Noodle","Pack","South Korea",5,456.39999999999998
2281,"Nissin","Demae Iccho Red Hot Seafood Flavou

2192,"Nyor Nyar","Penang White Curry Instant Noodle","Pack","Malaysia",5,438.39999999999998
2191,"Choumama","XO Scallops Sauce Dry Noodles","Pack","Taiwan",4,547.75
2190,"Myojo","Barikata Noukou Tonkotsu Shoyu","Bowl","Japan",4.75,461.05263157894734
2189,"Nissin","Demae Ramen Sesame Oil Flavour Cup","Cup","Hong Kong",4,547.25
2188,"Singa-Me","Instant Noodles Chicken Flavour","Cup","Australia",2.5,875.20000000000005
2187,"Pulmuone","Non-Fried Ramyun Noodle (Crab Flavor)","Pack","South Korea",5,437.39999999999998
2186,"Master Kong","Assorted Seafood Noodles","Pack","China",4,546.5
2185,"Nissin","Raoh Hot & Sour Ramen","Pack","Japan",5,437
2184,"President Rice","Coriander Instant Rice Noodles","Bowl","Thailand",2.75,794.18181818181813
2183,"MAMA","Instant Rice Noodles Moo Nam Tok Flavour","Pack","Thailand",3.5,623.71428571428567
2182,"MAMA","Instant Rice Vermicelli Clear Soup","Pack","Thailand",3.5,623.42857142857144
2181,"MAMA","Instant Rice Noodles Chicken Flavour","Pack","Thailand",4,5

...

540,"Nissin","Top Ramen Chicken","Pack","USA",3.25,166.15384615384616
539,"Nissin","Demae Ramen Chicken","Pack","Japan",3,179.66666666666666
538,"Lucky Me!","Lomi Seafood Vegetable","Pack","Philippines",0.5,1076
537,"Mama","Pad Kee Mao Drunken","Pack","Thailand",2,268.5
536,"Ottogi","Yeul Ramyun","Pack","South Korea",4.25,126.11764705882354
535,"Wai Wai","Chicken Flavor","Pack","Thailand",0.90000000000000002,594.44444444444446
534,"Mama","Pho Ga","Pack","Thailand",3.1000000000000001,172.25806451612902
533,"Paldo","Gomtang","Pack","South Korea",4.75,112.21052631578948
532,"Doll","Spicy Tomkotsu","Pack","Hong Kong",4.25,125.17647058823529
531,"Yum Yum","Beef","Pack","Thailand",3.25,163.38461538461539
530,"Mee Jang","Artificial Shrimp Tom yum","Bowl","Thailand",3.5,151.42857142857142
529,"Sapporo Ichiban","Hot Spicy Chicken","Pack","USA",2,264.5
528,"Unox","Good Noodles Vegetable","Pack","Holland",3.75,140.80000000000001
527,"Little Cook","Wheat Gluten Stewed Duck Flavoured","Bowl","Thail

428,"Kamfen","E Menm Chicken",,"China",3.75,114.13333333333334
427,"Koka","Mi Hai Cua Crab Flavor","Pack","Singapore",5,85.400000000000006
426,"Vifon","Tu quy Chicken","Pack","Vietnam",3,142
425,"Chikara","Shrimp Udon","Pack","USA",4.5,94.444444444444443
424,"Q","Noodle With Sesame Sauce","Pack","Taiwan",3.25,130.46153846153845
423,"Noodle Time","Shin Cup","Cup","South Korea",3,141
422,"Vifon","Mi Lau Thai Thai Stle","Bowl","Vietnam",4,105.5
421,"Myojo","Kakesoba Desse Soy Sauce","Bowl","Japan",2.75,153.09090909090909
420,"Sapporo Ichiban","Beef Flavorc","Cup","USA",3.5,120
419,"Myojo","Ramen Desse Shio","Bowl","Japan",4.25,98.588235294117652
418,"iNoodle","Udon Noodle Soup Oriental","Pack","Taiwan",3.75,111.46666666666667
417,"Vifon","Tu Quy Spicy Beef","Pack","Vietnam",3.5,119.14285714285714
416,"Sapporo Ichiban","Chicken Flavor","Cup","USA",3.75,110.93333333333334
415,"Ve Wong","Artificial Classic Beef","Bowl","Taiwan",3.75,110.66666666666667
414,"Doll","Artificial Beef Flavor","Pac

319,"Maruchan","Ramen Noodle Soup Oriental","Pack","USA",3.75,85.066666666666663
318,"Unif","Super Bowl Artificial Pickled Chilli Beef","Bowl","Taiwan",3.25,97.84615384615384
317,"Maruchan","Yakisoba Cheddar Cheese","Tray","USA",4,79.25
316,"Unif","Super Bowl Artificial Stewed Pork Chop","Bowl","Taiwan",3.5,90.285714285714292
315,"Unif","Super Bowl Artificial Spicy Beef","Bowl","Taiwan",2.75,114.54545454545455
314,"Paldo","Dosirac Mushroom","Tray","South Korea",2.5,125.59999999999999
313,"Paldo","Dosirac Shrimp","Tray","South Korea",4.25,73.647058823529406
312,"Lucky Me!","Supreme Bulalo Artificial Bone Marrow","Bowl","Philippines",1.75,178.28571428571428
311,"Vifon","Tasty Mushroom","Pack","Vietnam",2,155.5
310,"Lucky Me!","La Paz Batchoy Beef","Pack","Philippines",2.5,124
309,"Paldo","Dosirac Beef","Tray","South Korea",3.75,82.400000000000006
308,"Tradition","Imitation Chicken Vegetarian","Pack","USA",3.75,82.13333333333334
307,"Royal Umbrella","Vegetarian Tom Yum","Cup","Thailand",2

210,"Rocket Brand","Satoimo Noodles","Pack","China",2.5,84
209,"Nissin","Souper Meal Beef Flavor Minestrone","Bowl","USA",3,69.666666666666671
208,"Sao Tao","Ramen King Scallop Seafood","Pack","China",4.25,48.941176470588232
207,"Sapporo Ichiban","Original Flavor","Pack","USA",3.5,59.142857142857146
206,"Bon Go Jang","Deuchi Udon","Bowl","South Korea",3,68.666666666666671
205,"Sapporo Ichiban","Beef Flavor","Pack","USA",4.5,45.555555555555557
204,"Nongshim","Bowl Noodle Spicy Shrimp","Bowl","USA",2.5,81.599999999999994
203,"Nongshim","Bowl Noodle Hot & Spicy","Bowl","USA",2,101.5
202,"Nongshim","Bowl Noodle Cabbage Kimchi","Bowl","USA",3.25,62.153846153846153
201,"Nongshim","Bowl Noodle Beef Ginger","Bowl","USA",3.5,57.428571428571431
200,"Nongshim","Bowl Noodle Spicy Chicken","Bowl","USA",4.5,44.444444444444443
199,"Samyang","Chinese Soybean Paste","Pack","South Korea",2,99.5
198,"Shirakiku","Sanukiya Somen","Bowl","Japan",4.25,46.588235294117645
197,"Rhee Bros Assi","Rice Noodle With

99,"Ve Wong","Kung Fu Artificial Onion","Pack","Taiwan",2.75,36
98,"Unif / Tung-I","Chah Chiang","Pack","Taiwan",2.5,39.200000000000003
97,"Saigon Ve Wong","Kung Fu Mi Tom Chua Sour Shrimp","Pack","Vietnam",2.5,38.799999999999997
96,"Saigon Ve Wong","Kung Fu Artificial Pork Flavor","Pack","Vietnam",3.75,25.600000000000001
95,"Wei Lih","Artificial Beef Flavor","Pack","Taiwan",1,95
94,"Mama","Artificial Chicken","Pack","Thailand",2.5,37.600000000000001
93,"Payless","Xtra Big Original Pancit Canton","Pack","Philippines",4.5,20.666666666666668
92,"Fashion Food","Tom Yum Seafood Creamy","Bowl","Thailand",2,46
91,"Binh Tay","Mi Chay Mushroom","Pack","Vietnam",2.75,33.090909090909093
90,"Long Kow","Crystal Noodle Six Kinds Of Mushrooms","Pack","China",2.2999999999999998,39.130434782608695
89,"Unif","100 Artificial Stewed Pork Chop","Pack","Taiwan",3.2000000000000002,27.8125
88,"Paldo","South Korean Noodle Chicken","Pack","South Korea",3.25,27.076923076923077
87,"Wai Wai","Tom Yum Shrimp Cream

**[Builtin Variables](http://auriq.com/documentation/source/reference/manpages/aq_pp.html#eval)**
You can also use builtin variables. There are couple of them, and here we'll take a look at `$RowNum` and `$Random`.

Using `$RowNum`, we'll create a new integer column `row` and store the row number.

In [7]:
aq_pp -f,+1 $file -d $cols -eval i:row '$RowNum'

"reviewID","brand","variety","style","country","stars","row"
2580,"New Touch","T's Restaurant Tantanmen ","Cup","Japan",3.75,1
2579,"Just Way","Noodles Spicy Hot Sesame Spicy Hot Sesame Guan-miao Noodles","Pack","Taiwan",1,2
2578,"Nissin","Cup Noodles Chicken Vegetable","Cup","USA",2.25,3
2577,"Wei Lih","GGE Ramen Snack Tomato Flavor","Pack","Taiwan",2.75,4
2576,"Ching's Secret","Singapore Curry","Pack","India",3.75,5
2575,"Samyang Foods","Kimchi song Song Ramen","Pack","South Korea",4.75,6
2574,"Acecook","Spice Deli Tantan Men With Cilantro","Cup","Japan",4,7
2573,"Ikeda Shoku","Nabeyaki Kitsune Udon","Tray","Japan",3.75,8
2572,"Ripe'n'Dry","Hokkaido Soy Sauce Ramen","Pack","Japan",0.25,9
2571,"KOKA","The Original Spicy Stir-Fried Noodles","Pack","Singapore",2.5,10
2570,"Tao Kae Noi","Creamy tom Yum Kung Flavour","Pack","Thailand",5,11
2569,"Yamachan","Yokohama Tonkotsu Shoyu","Pack","USA",5,12
2568,"Nongshim","Mr. Bibim Stir-Fried Kimchi Flavor","Pack","South Korea",4.25,13
2567,"Nis

2465,"Nissin","Cup Noodles XO Sauce Seafood","Cup","China",3.75,115
2464,"Wang","Extreme Spicy Hot Chicken Flavor Udon","Bowl","South Korea",2,116
2463,"Nissin","Gekikara Ramen Rasa Pedas","Pack","Indonesia",3.25,117
2462,"TTL","Sesame Oil Chicken Noodle With Rice Wine","Bowl","Taiwan",5,118
2461,"Fujiwara","Hokkaido Hakodate Shio Ramen","Pack","Japan",4.5,119
2460,"Daifuku","Katsuo Bowl Udon","Bowl","USA",5,120
2459,"Nissin","Cup Noodles MUG Shoyu & Seafood","Pack","Japan",3.5,121
2457,"KOKA","Instant Noodles Chicken Satay Flavour","Pack","Singapore",3.5,122
2456,"KOKA","Signature Spicy Singapore Fried Noodles Instant Noodles","Pack","Singapore",4.25,123
2455,"KOKA","Creamy Soup With Crushed Noodles Sweet Corn Flavor","Cup","Singapore",5,124
2454,"KOKA","Silk Beef Pho Flavor Instant Rice Fettuccine","Bowl","Singapore",3.25,125
2453,"KOKA","Delight Spicy Sesame Flavor Instant Non-Fried Noodles","Pack","Singapore",4.5,126
2452,"KOKA","Signature Laksa Singapura Instant Noodles","Bowl","S

2361,"Nissin","Demae Iccho Chicken Flavour Instant Noodle (Bowl Noodle)","Bowl","Hong Kong",5,218
2360,"Wai Wai","Instant Noodles Artificial Chicken Flavoured","Pack","India",2,219
2359,"Daikoku","Kitsune Udon","Cup","Japan",2,220
2358,"Sichuan Baijia","Pickled Cabbage Flavor Instant Vermicelli","Bowl","China",4,221
2357,"Kamfen","Noodle King Artificial Wonton Soup Flavored","Bowl","Hong Kong",5,222
2356,"Nongshim","Spicy Shrimp Cup Noodle","Cup","South Korea",4,223
2355,"Marutai","Nagasaki Agodashi Shoyu Ramen","Pack","Japan",4,224
2354,"Vifon","Viet Cuisine Bun Rieu Cua Sour Crab Soup Instant Rice Vermicelli","Bowl","Vietnam",5,225
2353,"Nissin","Cup Noodles Rich Garlic, Egg Yolk & Oxtail","Cup","Japan",4,226
2352,"Uni-President","Minced Pork Flavor Instant Noodles","Pack","Taiwan",1,227
2351,"1 To 3 Noodles","Chatpat Masala","Pack","India",4,228
2350,"Nissin","Cup Noodles BIG XO Sauce Seafood Flavour","Cup","Hong Kong",4,229
2349,"Fantastic","Noodles Chicken & Corn Flavour","Cup","A

2257,"Fuku","Tom Yam Soup Instant Noodle","Pack","Hong Kong",2.2999999999999998,322
2256,"Goku-Uma","Ramen Noodles Artificially Flavored Tonkotsu","Bowl","USA",3,323
2255,"Daikoku","Hiroshima Flavor Yakisoba","Tray","Japan",5,324
2254,"Nissin","Disney Cuties Instant Noodle Seaweed Flavour","Cup","Thailand",3,325
2253,"Singa-Me","Instant Noodles Mi Goreng Flavour","Cup","Australia",4,326
2252,"New Touch","Sugomen Sano Ramen","Bowl","Japan",5,327
2251,"Ottogi","Ramyon Sabor Y Camaron","Pack","Mexico",4,328
2250,"Nissin","Demae Iccho Spicy XO Sauce Seafood Flavour","Pack","Hong Kong",4,329
2249,"Love Cook","Sun Dried Noodle - Fruity Soy Bean Paste","Pack","Taiwan",4,330
2248,"Nongshim","Shin Noodle Soup","Cup","USA",5,331
2247,"Nissin","Sapporo Noukou Miso Ramen","Bowl","Japan",5,332
2246,"Dream Kitchen","Soy Sauce","Cup","USA",4,333
2245,"Nissin","Vietnamese Chicken Coriander Flavour Vermicelli","Cup","Hong Kong",4,334
2244,"Kang Shi Fu","Artificial Spicy Beef Flavour","Pack","Hong Kong"

2149,"Jingqi","Aloe Vera Guan Mian Cyanobacteria Noodle With Ginger Oil Sauce","Pack","Taiwan",5,430
2148,"Jingqi","Black Eyes Bean Pumpkin Noodles With Basil Sauce","Pack","Taiwan",5,431
2147,"Maruchan","Instant Lunch Pork Flavor Ramen Noodles With Vegetables","Cup","USA",3,432
2146,"Sapporo Ichiban","Kun Ramen Sumo-kuchikin (Smoked Chicken)","Bowl","Japan",3,433
2145,"Dream Kitchen","Tonkotsu","Cup","USA",3,434
2144,"Nissin","Lamen Sabor Picanha","Pack","Brazil",4,435
2143,"Thien Houng Foods","Lemon Chicken Flavour Instant Noodles","Pack","Vietnam",3,436
2142,"Maruchan","Bowl Taste Of Asia Miso Chicken Flavor Spicy Miso Ramen","Bowl","USA",4,437
2141,"Nissin","Men Shokunin Kaoruyasai Shio","Bowl","Japan",5,438
2140,"Wang","Rice Noodle Seafood Flavour","Bowl","South Korea",5,439
2139,"Fuku","Superior Soup Instant Noodles (Mfg in Thailand)","Pack","Hong Kong",4,440
2138,"Fuku","Superior Soup Instant Noodles (Mfg in Vietnam)","Pack","Hong Kong",4,441
2137,"Nissin","Demae Ramen Spicy Cur

...

612,"Maruchan","Yakisoba Spicy Chicken","Tray","USA",1.75,1966
611,"Batchelors","Super Noodles Low Fat Sweet Thai Chilli","Pack","UK",2,1967
610,"Wei Wei","""A"" Series Artificial Chicken","Pack","Taiwan",3.25,1968
609,"Wei Wei","""A"" Series Artificial Hot Beef","Pack","Taiwan",3.75,1969
608,"Koka","Spicy Black Pepper","Pack","Singapore",5,1970
607,"Pot Noodle","Beef Tomato","Cup","UK",1.5,1971
606,"Six Fortune","U-Dong","Pack","South Korea",4,1972
605,"Little Cook","Artificial Onion","Cup","Thailand",3.5,1973
604,"Batchelors","Super Noodles Low Fat Chicken Herb","Pack","UK",2,1974
603,"Wei Lih","Goog Good Eat Super Ramen Hot Chili","Pack","Taiwan",3.75,1975
602,"Nongshim","Oolongmen Artificial Seafood","Cup","South Korea",3.5,1976
601,"Wei Chuan","Tomato Vegetables","Pack","Taiwan",4,1977
600,"Maruchan","Ramen Noodle Soup Beef","Pack","USA",3.25,1978
599,"Lucky Me!","Pancit Canton Extra Hot Chili","Pack","Philippines",4,1979
598,"Little Cook","Vegetarian","Cup","Thailand",3.25,1980
5

483,"Itomen","Yellow Buckwheat","Bowl","Japan",4.5,2095
482,"Menraku","Shio","Bowl","Japan",4.5,2096
481,"Thai Kitchen","Roasted Garlic Rice Noodle","Bowl","USA",3.75,2097
480,"Thai Kitchen","Mushroom Rice Noodle","Bowl","USA",3.25,2098
479,"Nongshim","Potato Noodle Soup","Pack","South Korea",3.75,2099
478,"Vedan","Mushroom & Artificial Cuttlefish","Pack","Taiwan",1.75,2100
477,"Mr. Udon","Malaysian Beef Udon","Pack","South Korea",3,2101
476,"Annie Chun's","Teriyaki Noodle","Bowl","USA",1.75,2102
475,"Gefen","Vegetable","Pack","USA",3.25,2103
474,"Saigon Ve Wong","Kung Fu Rice Noodle Shrimp Crab ","Bowl","Vietnam",3.75,2104
473,"Myojo","Ramen-Desse Miso Soybean Paste","Bowl","Japan",4.5,2105
472,"Thai Kitchen","Spring Onion Rice Noodle","Bowl","USA",3.75,2106
471,"Koyo","Lemongrass ginger","Pack","USA",1.5,2107
470,"Maruchan","Ramen Noodle Soup Chicken Mushroom","Pack","USA",3.25,2108
469,"Gefen","Chicken","Pack","USA",3.5,2109
468,"Paldo","Snack Noodle slightly Hot","Pack","South Kore

358,"Doll","Spicy Artificial Pork","Pack","Hong Kong",3.75,2220
357,"Payless","Xtra Big Kalamansi Pancit Canton","Pack","Philippines",4,2221
356,"Maruchan","Ramen Noodle Soup Roast Beef","Pack","USA",2.75,2222
355,"Nongshim","Noodle Lite 275 Udon Mushroom","Bowl","South Korea",2.5,2223
354,"Maruchan","Instant Lunch Chicken 35% Less Sodium","Cup","USA",3,2224
353,"Nissin","Top Ramen Chili","Pack","USA",3.25,2225
352,"Maruchan","Ramen Noodle Soup Chili","Pack","USA",2.75,2226
351,"Nissin","Top Ramen Picante Beef","Pack","USA",3,2227
350,"Nissin","Souper Meal Tomato Garlic Shrimp","Bowl","USA",3.5,2228
349,"Ottogi","Ramen Bokki","Bowl","South Korea",3.25,2229
348,"Paldo","Bowl Noodle Kimchi","Bowl","South Korea",3.5,2230
347,"Sunny Maid","Mi Chay Vegetarian","Pack","Taiwan",1.75,2231
346,"Paldo","Bowl Noodle Spicy Artificial Chicken","Bowl","South Korea",3.125,2232
345,"Myojo","Charumera Shoyu","Pack","Japan",3.5,2233
344,"Paldo","Stirfried Kimchi","Pack","South Korea",3.75,2234
343,"Supe

233,"Koka","Tom Yum Rice Noodles","Bowl","Singapore",3.5,2345
232,"Indomie","Mi Goreng Jumbo Beef","Pack","Indonesia",5,2346
231,"Indomie","Onion Chicken","Pack","Indonesia",4.5,2347
230,"Lucky Me!","Supreme Instant Mami Noodles With Free Crackers","Bowl","Philippines",3.75,2348
229,"Myojo","Udon Chicken","Pack","Japan",3.75,2349
228,"Vifon","Pho Ga Instant rice Noodle","Bowl","Vietnam",2,2350
227,"Quickchow","Bihon Guisado Rice Noodle","Pack","Philippines",1.75,2351
226,"Nissin","Chow Noodles Cheddar Cheese","Tray","USA",4,2352
225,"Ottogi","Snack Ramyon","Pack","South Korea",3.75,2353
224,"Mama","Instant Bowl Noodles Artificial Pork","Bowl","Thailand",2.75,2354
223,"Koka","Laksa Singapura","Bowl","Singapore",4.25,2355
222,"Myojo","Udon Crab","Pack","Japan",3,2356
221,"Nissin","Top Ramen Beef","Pack","USA",3.75,2357
220,"Wei Lih","Good Good Eat Super Ramen Original","Pack","Taiwan",3,2358
219,"Six Fortune","Tom Yam Instant soup Noodle","Pack","South Korea",2.75,2359
218,"Maruchan","Ya

106,"GreeNoodle","Tom Yum","Pack","Thailand",0.5,2472
105,"Indomie","Special Fried Curly Noodle","Pack","Indonesia",5,2473
104,"Maruchan","Ramen Noodle Soup Chicken","Pack","USA",2,2474
103,"Ve Wong","Vegetarian Flavor","Pack","Taiwan",3,2475
102,"Indomie","Soto Mie","Pack","Indonesia",1.5,2476
101,"Ve Wong","Artificial Sesame Chicken","Pack","Taiwan",4.25,2477
100,"Unif / Tung-I","Artificial Chinese Beef","Pack","Taiwan",2.8500000000000001,2478
99,"Ve Wong","Kung Fu Artificial Onion","Pack","Taiwan",2.75,2479
98,"Unif / Tung-I","Chah Chiang","Pack","Taiwan",2.5,2480
97,"Saigon Ve Wong","Kung Fu Mi Tom Chua Sour Shrimp","Pack","Vietnam",2.5,2481
96,"Saigon Ve Wong","Kung Fu Artificial Pork Flavor","Pack","Vietnam",3.75,2482
95,"Wei Lih","Artificial Beef Flavor","Pack","Taiwan",1,2483
94,"Mama","Artificial Chicken","Pack","Thailand",2.5,2484
93,"Payless","Xtra Big Original Pancit Canton","Pack","Philippines",4.5,2485
92,"Fashion Food","Tom Yum Seafood Creamy","Bowl","Thailand",2,2486
91

Since we are skipping the header row with `-f,+1` option, we'll correct the row numbers by addding 1 to each row number.

In [6]:
aq_pp -f,+1 $file -d $cols -eval i:row '$RowNum +1'

"reviewID","brand","variety","style","country","stars","row"
2580,"New Touch","T's Restaurant Tantanmen ","Cup","Japan",3.75,2
2579,"Just Way","Noodles Spicy Hot Sesame Spicy Hot Sesame Guan-miao Noodles","Pack","Taiwan",1,3
2578,"Nissin","Cup Noodles Chicken Vegetable","Cup","USA",2.25,4
2577,"Wei Lih","GGE Ramen Snack Tomato Flavor","Pack","Taiwan",2.75,5
2576,"Ching's Secret","Singapore Curry","Pack","India",3.75,6
2575,"Samyang Foods","Kimchi song Song Ramen","Pack","South Korea",4.75,7
2574,"Acecook","Spice Deli Tantan Men With Cilantro","Cup","Japan",4,8
2573,"Ikeda Shoku","Nabeyaki Kitsune Udon","Tray","Japan",3.75,9
2572,"Ripe'n'Dry","Hokkaido Soy Sauce Ramen","Pack","Japan",0.25,10
2571,"KOKA","The Original Spicy Stir-Fried Noodles","Pack","Singapore",2.5,11
2570,"Tao Kae Noi","Creamy tom Yum Kung Flavour","Pack","Thailand",5,12
2569,"Yamachan","Yokohama Tonkotsu Shoyu","Pack","USA",5,13
2568,"Nongshim","Mr. Bibim Stir-Fried Kimchi Flavor","Pack","South Korea",4.25,14
2567,"Ni

2465,"Nissin","Cup Noodles XO Sauce Seafood","Cup","China",3.75,116
2464,"Wang","Extreme Spicy Hot Chicken Flavor Udon","Bowl","South Korea",2,117
2463,"Nissin","Gekikara Ramen Rasa Pedas","Pack","Indonesia",3.25,118
2462,"TTL","Sesame Oil Chicken Noodle With Rice Wine","Bowl","Taiwan",5,119
2461,"Fujiwara","Hokkaido Hakodate Shio Ramen","Pack","Japan",4.5,120
2460,"Daifuku","Katsuo Bowl Udon","Bowl","USA",5,121
2459,"Nissin","Cup Noodles MUG Shoyu & Seafood","Pack","Japan",3.5,122
2457,"KOKA","Instant Noodles Chicken Satay Flavour","Pack","Singapore",3.5,123
2456,"KOKA","Signature Spicy Singapore Fried Noodles Instant Noodles","Pack","Singapore",4.25,124
2455,"KOKA","Creamy Soup With Crushed Noodles Sweet Corn Flavor","Cup","Singapore",5,125
2454,"KOKA","Silk Beef Pho Flavor Instant Rice Fettuccine","Bowl","Singapore",3.25,126
2453,"KOKA","Delight Spicy Sesame Flavor Instant Non-Fried Noodles","Pack","Singapore",4.5,127
2452,"KOKA","Signature Laksa Singapura Instant Noodles","Bowl","S

2361,"Nissin","Demae Iccho Chicken Flavour Instant Noodle (Bowl Noodle)","Bowl","Hong Kong",5,219
2360,"Wai Wai","Instant Noodles Artificial Chicken Flavoured","Pack","India",2,220
2359,"Daikoku","Kitsune Udon","Cup","Japan",2,221
2358,"Sichuan Baijia","Pickled Cabbage Flavor Instant Vermicelli","Bowl","China",4,222
2357,"Kamfen","Noodle King Artificial Wonton Soup Flavored","Bowl","Hong Kong",5,223
2356,"Nongshim","Spicy Shrimp Cup Noodle","Cup","South Korea",4,224
2355,"Marutai","Nagasaki Agodashi Shoyu Ramen","Pack","Japan",4,225
2354,"Vifon","Viet Cuisine Bun Rieu Cua Sour Crab Soup Instant Rice Vermicelli","Bowl","Vietnam",5,226
2353,"Nissin","Cup Noodles Rich Garlic, Egg Yolk & Oxtail","Cup","Japan",4,227
2352,"Uni-President","Minced Pork Flavor Instant Noodles","Pack","Taiwan",1,228
2351,"1 To 3 Noodles","Chatpat Masala","Pack","India",4,229
2350,"Nissin","Cup Noodles BIG XO Sauce Seafood Flavour","Cup","Hong Kong",4,230
2349,"Fantastic","Noodles Chicken & Corn Flavour","Cup","A

2257,"Fuku","Tom Yam Soup Instant Noodle","Pack","Hong Kong",2.2999999999999998,323
2256,"Goku-Uma","Ramen Noodles Artificially Flavored Tonkotsu","Bowl","USA",3,324
2255,"Daikoku","Hiroshima Flavor Yakisoba","Tray","Japan",5,325
2254,"Nissin","Disney Cuties Instant Noodle Seaweed Flavour","Cup","Thailand",3,326
2253,"Singa-Me","Instant Noodles Mi Goreng Flavour","Cup","Australia",4,327
2252,"New Touch","Sugomen Sano Ramen","Bowl","Japan",5,328
2251,"Ottogi","Ramyon Sabor Y Camaron","Pack","Mexico",4,329
2250,"Nissin","Demae Iccho Spicy XO Sauce Seafood Flavour","Pack","Hong Kong",4,330
2249,"Love Cook","Sun Dried Noodle - Fruity Soy Bean Paste","Pack","Taiwan",4,331
2248,"Nongshim","Shin Noodle Soup","Cup","USA",5,332
2247,"Nissin","Sapporo Noukou Miso Ramen","Bowl","Japan",5,333
2246,"Dream Kitchen","Soy Sauce","Cup","USA",4,334
2245,"Nissin","Vietnamese Chicken Coriander Flavour Vermicelli","Cup","Hong Kong",4,335
2244,"Kang Shi Fu","Artificial Spicy Beef Flavour","Pack","Hong Kong"

2149,"Jingqi","Aloe Vera Guan Mian Cyanobacteria Noodle With Ginger Oil Sauce","Pack","Taiwan",5,431
2148,"Jingqi","Black Eyes Bean Pumpkin Noodles With Basil Sauce","Pack","Taiwan",5,432
2147,"Maruchan","Instant Lunch Pork Flavor Ramen Noodles With Vegetables","Cup","USA",3,433
2146,"Sapporo Ichiban","Kun Ramen Sumo-kuchikin (Smoked Chicken)","Bowl","Japan",3,434
2145,"Dream Kitchen","Tonkotsu","Cup","USA",3,435
2144,"Nissin","Lamen Sabor Picanha","Pack","Brazil",4,436
2143,"Thien Houng Foods","Lemon Chicken Flavour Instant Noodles","Pack","Vietnam",3,437
2142,"Maruchan","Bowl Taste Of Asia Miso Chicken Flavor Spicy Miso Ramen","Bowl","USA",4,438
2141,"Nissin","Men Shokunin Kaoruyasai Shio","Bowl","Japan",5,439
2140,"Wang","Rice Noodle Seafood Flavour","Bowl","South Korea",5,440
2139,"Fuku","Superior Soup Instant Noodles (Mfg in Thailand)","Pack","Hong Kong",4,441
2138,"Fuku","Superior Soup Instant Noodles (Mfg in Vietnam)","Pack","Hong Kong",4,442
2137,"Nissin","Demae Ramen Spicy Cur

...

612,"Maruchan","Yakisoba Spicy Chicken","Tray","USA",1.75,1967
611,"Batchelors","Super Noodles Low Fat Sweet Thai Chilli","Pack","UK",2,1968
610,"Wei Wei","""A"" Series Artificial Chicken","Pack","Taiwan",3.25,1969
609,"Wei Wei","""A"" Series Artificial Hot Beef","Pack","Taiwan",3.75,1970
608,"Koka","Spicy Black Pepper","Pack","Singapore",5,1971
607,"Pot Noodle","Beef Tomato","Cup","UK",1.5,1972
606,"Six Fortune","U-Dong","Pack","South Korea",4,1973
605,"Little Cook","Artificial Onion","Cup","Thailand",3.5,1974
604,"Batchelors","Super Noodles Low Fat Chicken Herb","Pack","UK",2,1975
603,"Wei Lih","Goog Good Eat Super Ramen Hot Chili","Pack","Taiwan",3.75,1976
602,"Nongshim","Oolongmen Artificial Seafood","Cup","South Korea",3.5,1977
601,"Wei Chuan","Tomato Vegetables","Pack","Taiwan",4,1978
600,"Maruchan","Ramen Noodle Soup Beef","Pack","USA",3.25,1979
599,"Lucky Me!","Pancit Canton Extra Hot Chili","Pack","Philippines",4,1980
598,"Little Cook","Vegetarian","Cup","Thailand",3.25,1981
5

483,"Itomen","Yellow Buckwheat","Bowl","Japan",4.5,2096
482,"Menraku","Shio","Bowl","Japan",4.5,2097
481,"Thai Kitchen","Roasted Garlic Rice Noodle","Bowl","USA",3.75,2098
480,"Thai Kitchen","Mushroom Rice Noodle","Bowl","USA",3.25,2099
479,"Nongshim","Potato Noodle Soup","Pack","South Korea",3.75,2100
478,"Vedan","Mushroom & Artificial Cuttlefish","Pack","Taiwan",1.75,2101
477,"Mr. Udon","Malaysian Beef Udon","Pack","South Korea",3,2102
476,"Annie Chun's","Teriyaki Noodle","Bowl","USA",1.75,2103
475,"Gefen","Vegetable","Pack","USA",3.25,2104
474,"Saigon Ve Wong","Kung Fu Rice Noodle Shrimp Crab ","Bowl","Vietnam",3.75,2105
473,"Myojo","Ramen-Desse Miso Soybean Paste","Bowl","Japan",4.5,2106
472,"Thai Kitchen","Spring Onion Rice Noodle","Bowl","USA",3.75,2107
471,"Koyo","Lemongrass ginger","Pack","USA",1.5,2108
470,"Maruchan","Ramen Noodle Soup Chicken Mushroom","Pack","USA",3.25,2109
469,"Gefen","Chicken","Pack","USA",3.5,2110
468,"Paldo","Snack Noodle slightly Hot","Pack","South Kore

358,"Doll","Spicy Artificial Pork","Pack","Hong Kong",3.75,2221
357,"Payless","Xtra Big Kalamansi Pancit Canton","Pack","Philippines",4,2222
356,"Maruchan","Ramen Noodle Soup Roast Beef","Pack","USA",2.75,2223
355,"Nongshim","Noodle Lite 275 Udon Mushroom","Bowl","South Korea",2.5,2224
354,"Maruchan","Instant Lunch Chicken 35% Less Sodium","Cup","USA",3,2225
353,"Nissin","Top Ramen Chili","Pack","USA",3.25,2226
352,"Maruchan","Ramen Noodle Soup Chili","Pack","USA",2.75,2227
351,"Nissin","Top Ramen Picante Beef","Pack","USA",3,2228
350,"Nissin","Souper Meal Tomato Garlic Shrimp","Bowl","USA",3.5,2229
349,"Ottogi","Ramen Bokki","Bowl","South Korea",3.25,2230
348,"Paldo","Bowl Noodle Kimchi","Bowl","South Korea",3.5,2231
347,"Sunny Maid","Mi Chay Vegetarian","Pack","Taiwan",1.75,2232
346,"Paldo","Bowl Noodle Spicy Artificial Chicken","Bowl","South Korea",3.125,2233
345,"Myojo","Charumera Shoyu","Pack","Japan",3.5,2234
344,"Paldo","Stirfried Kimchi","Pack","South Korea",3.75,2235
343,"Supe

233,"Koka","Tom Yum Rice Noodles","Bowl","Singapore",3.5,2346
232,"Indomie","Mi Goreng Jumbo Beef","Pack","Indonesia",5,2347
231,"Indomie","Onion Chicken","Pack","Indonesia",4.5,2348
230,"Lucky Me!","Supreme Instant Mami Noodles With Free Crackers","Bowl","Philippines",3.75,2349
229,"Myojo","Udon Chicken","Pack","Japan",3.75,2350
228,"Vifon","Pho Ga Instant rice Noodle","Bowl","Vietnam",2,2351
227,"Quickchow","Bihon Guisado Rice Noodle","Pack","Philippines",1.75,2352
226,"Nissin","Chow Noodles Cheddar Cheese","Tray","USA",4,2353
225,"Ottogi","Snack Ramyon","Pack","South Korea",3.75,2354
224,"Mama","Instant Bowl Noodles Artificial Pork","Bowl","Thailand",2.75,2355
223,"Koka","Laksa Singapura","Bowl","Singapore",4.25,2356
222,"Myojo","Udon Crab","Pack","Japan",3,2357
221,"Nissin","Top Ramen Beef","Pack","USA",3.75,2358
220,"Wei Lih","Good Good Eat Super Ramen Original","Pack","Taiwan",3,2359
219,"Six Fortune","Tom Yam Instant soup Noodle","Pack","South Korea",2.75,2360
218,"Maruchan","Ya

106,"GreeNoodle","Tom Yum","Pack","Thailand",0.5,2473
105,"Indomie","Special Fried Curly Noodle","Pack","Indonesia",5,2474
104,"Maruchan","Ramen Noodle Soup Chicken","Pack","USA",2,2475
103,"Ve Wong","Vegetarian Flavor","Pack","Taiwan",3,2476
102,"Indomie","Soto Mie","Pack","Indonesia",1.5,2477
101,"Ve Wong","Artificial Sesame Chicken","Pack","Taiwan",4.25,2478
100,"Unif / Tung-I","Artificial Chinese Beef","Pack","Taiwan",2.8500000000000001,2479
99,"Ve Wong","Kung Fu Artificial Onion","Pack","Taiwan",2.75,2480
98,"Unif / Tung-I","Chah Chiang","Pack","Taiwan",2.5,2481
97,"Saigon Ve Wong","Kung Fu Mi Tom Chua Sour Shrimp","Pack","Vietnam",2.5,2482
96,"Saigon Ve Wong","Kung Fu Artificial Pork Flavor","Pack","Vietnam",3.75,2483
95,"Wei Lih","Artificial Beef Flavor","Pack","Taiwan",1,2484
94,"Mama","Artificial Chicken","Pack","Thailand",2.5,2485
93,"Payless","Xtra Big Original Pancit Canton","Pack","Philippines",4.5,2486
92,"Fashion Food","Tom Yum Seafood Creamy","Bowl","Thailand",2,2487
91

In [8]:
aq_pp -f,+1 $file -d $cols -eval i:row '$random'

"reviewID","brand","variety","style","country","stars","row"
2580,"New Touch","T's Restaurant Tantanmen ","Cup","Japan",3.75,476707713
2579,"Just Way","Noodles Spicy Hot Sesame Spicy Hot Sesame Guan-miao Noodles","Pack","Taiwan",1,1186278907
2578,"Nissin","Cup Noodles Chicken Vegetable","Cup","USA",2.25,505671508
2577,"Wei Lih","GGE Ramen Snack Tomato Flavor","Pack","Taiwan",2.75,2137716191
2576,"Ching's Secret","Singapore Curry","Pack","India",3.75,936145377
2575,"Samyang Foods","Kimchi song Song Ramen","Pack","South Korea",4.75,1215825599
2574,"Acecook","Spice Deli Tantan Men With Cilantro","Cup","Japan",4,589265238
2573,"Ikeda Shoku","Nabeyaki Kitsune Udon","Tray","Japan",3.75,924859463
2572,"Ripe'n'Dry","Hokkaido Soy Sauce Ramen","Pack","Japan",0.25,1182112391
2571,"KOKA","The Original Spicy Stir-Fried Noodles","Pack","Singapore",2.5,899065992
2570,"Tao Kae Noi","Creamy tom Yum Kung Flavour","Pack","Thailand",5,1942841066
2569,"Yamachan","Yokohama Tonkotsu Shoyu","Pack","USA",5,170

2475,"Maruchan","Seimen Red Spicy Dandan Men","Bowl","Japan",4.75,1202724756
2474,"Ottogi","Ppushu Ppushu Noodle Snack Chilli Cheese Flavor","Pack","South Korea",4.25,846614141
2473,"Tokyo Noodle","Mini Instant Noodle Mild Curry Flavor","Pack","Japan",2,959122900
2472,"GGE","Noodle Snack Wheat Cracks Seaweed Flavor","Pack","Taiwan",3.5,1819186785
2471,"Nissin","Cup Noodles Mini Seafood Flavour","Cup","Hong Kong",5,353633592
2470,"Myojo","Udon Japanese Style Noodles With Soup Base Hot & Sour Flavor","Pack","USA",3.75,1401330879
2469,"KOKA","Signature Curry Flavor Instant Noodles","Cup","Singapore",3.5,2131759629
2468,"Acecook","Dossari Yasai Chanpon","Cup","Japan",4.25,1633316224
2467,"Paldo","Volcano Curry Kkokkomyun","Pack","South Korea",3.5,32640725
2466,"Sau Tao","Instant Noodle King Pepper Duck Flavour Soup","Pack","Hong Kong",4,1782949768
2465,"Nissin","Cup Noodles XO Sauce Seafood","Cup","China",3.75,936017334
2464,"Wang","Extreme Spicy Hot Chicken Flavor Udon","Bowl","South Kore

2378,"Sau Tao","Non-Fried Mix Noodle Black Pepper XO Sauce Flavoured","Pack","Hong Kong",4.5,1778541711
2377,"MyKuali","Penang White Curry Instant Noodle (Japan Version)","Box","Malaysia",5,97370123
2376,"Nissin","Disney Cuties Instant Chicken With Corn Flavour","Cup","Thailand",3.25,519852486
2375,"Uni-President","Man Han Feast Braised Pork Flavor Instant Noodles","Bowl","Taiwan",5,1045703580
2374,"World O' Noodle","Mi Goreng Garlic Flavour Fried Noodles","Pack","Indonesia",4.5,1905447453
2373,"Nissin","Top Ramen Atta Nooldes Masala","Pack","India",4.5,1691040598
2372,"Liu Quan","Instant Spicy Rice Noodle","Bowl","China",4,1885564994
2371,"Daikoku","Tanuki Soba","Cup","Japan",3.5,1092130023
2370,"Samyang Foods","Zzaldduck Buldak Bokkeummyun Snack","Pack","South Korea",4.5,426621967
2369,"Samyang Foods","Curry Buldak Bokkeummyun","Pack","South Korea",4.25,1646064863
2368,"Samyang Foods","Cool/Ice Buldak Bokkeummyun","Pack","South Korea",3.75,1066490951
2367,"Samyang Foods","2x Spicy Ha

2282,"Samyang Foods","Gold Jjamppong Fried Noodle","Pack","South Korea",5,1591824974
2281,"Nissin","Demae Iccho Red Hot Seafood Flavour Instant Noodle","Cup","Hong Kong",4,790586858
2280,"Sichuan Baijia","Chongqing Noodles Hot & Sour Flavor","Pack","China",4,1256130775
2279,"Nissin","Demae Iccho Spicy Tonkotsu Flavour Instant Noodle (Bowl Noodle)","Bowl","Hong Kong",5,1588880676
2278,"Nissin","Raoh Tantanmen","Pack","Japan",5,742480296
2277,"Goku-Uma","Ramen Noodles Soy Sauce Flavor","Bowl","USA",3,951529044
2276,"Nissin","Cup Noodles With Shrimp Ramen Noodle Soup (New Recipe)","Cup","USA",3,1454554224
2275,"JML","Hot & Sour Beef Noodles","Pack","China",4,1971677191
2274,"Suimin","Noodles With Braised Beef Flavour","Cup","Australia",4,1168111616
2273,"Uni-President","A Q Bucket Noodle Korean Kimchi Flavor","Bowl","Taiwan",4,1886355105
2272,"Nissin","Demae Iccho Spicy Kimchi Pot Flavour Instant Noodle (Bowl Noodle)","Bowl","Hong Kong",4,1046765657
2271,"Sichuan Baijia","Chongqing Noodle

2182,"MAMA","Instant Rice Vermicelli Clear Soup","Pack","Thailand",3.5,2124314928
2181,"MAMA","Instant Rice Noodles Chicken Flavour","Pack","Thailand",4,1996413166
2180,"MAMA","Pho Ga Rice Noodles With Artificial Chicken Flavour","Bowl","Thailand",3,529913134
2179,"MAMA","Oriental Kitchen Instant Rice Vermicelli In Gravy","Pack","Thailand",5,1783779417
2178,"MAMA","Instant Tom Yam Rice Noodles","Pack","Thailand",4,133276800
2177,"MAMA","Instant Rice Noodles Vegetable Flavour","Pack","Thailand",4,617849477
2176,"MAMA","Instant Rice Vermicelli Yentafo Tom Yam Mohfai","Pack","Thailand",5,702899460
2175,"MAMA","Instant Flat Noodles Clear Soup","Pack","Thailand",4,604070170
2174,"MAMA","Cup Rice Noodles Tom Saab Flavour","Cup","Thailand",5,1031794062
2173,"MAMA","Instant Rice Noodles Stew Beef Flavour","Pack","Thailand",4,63597325
2172,"MAMA","Instant Rice Noodles Phnom Penh Style","Pack","Thailand",4,1414981446
2171,"MAMA","Instant Flat Noodles Tom Yam Flavour","Pack","Thailand",4,90519803

...

545,"Golden Wheat","South Korean Style Spicy Beef","Pack","China",3.5,574521562
544,"Unox","Good Noodles Chicken","Pack","Holland",3.5,1247772860
543,"Menraku","Curry Udon","Bowl","Japan",4.75,923619782
542,"Mi Sedaap","Mi SupArtificial Chicken Special","Pack","Indonesia",3,836747498
541,"Yum Yum","Chicken","Pack","Thailand",3.5,2114637231
540,"Nissin","Top Ramen Chicken","Pack","USA",3.25,354102595
539,"Nissin","Demae Ramen Chicken","Pack","Japan",3,1217036361
538,"Lucky Me!","Lomi Seafood Vegetable","Pack","Philippines",0.5,540609556
537,"Mama","Pad Kee Mao Drunken","Pack","Thailand",2,140239556
536,"Ottogi","Yeul Ramyun","Pack","South Korea",4.25,218140688
535,"Wai Wai","Chicken Flavor","Pack","Thailand",0.90000000000000002,1866027792
534,"Mama","Pho Ga","Pack","Thailand",3.1000000000000001,1073331903
533,"Paldo","Gomtang","Pack","South Korea",4.75,3894258
532,"Doll","Spicy Tomkotsu","Pack","Hong Kong",4.25,743659124
531,"Yum Yum","Beef","Pack","Thailand",3.25,137214854
530,"Mee Jan

426,"Vifon","Tu quy Chicken","Pack","Vietnam",3,1012159227
425,"Chikara","Shrimp Udon","Pack","USA",4.5,1794800572
424,"Q","Noodle With Sesame Sauce","Pack","Taiwan",3.25,2098986787
423,"Noodle Time","Shin Cup","Cup","South Korea",3,1851883905
422,"Vifon","Mi Lau Thai Thai Stle","Bowl","Vietnam",4,2116138392
421,"Myojo","Kakesoba Desse Soy Sauce","Bowl","Japan",2.75,1819144765
420,"Sapporo Ichiban","Beef Flavorc","Cup","USA",3.5,320058368
419,"Myojo","Ramen Desse Shio","Bowl","Japan",4.25,128556971
418,"iNoodle","Udon Noodle Soup Oriental","Pack","Taiwan",3.75,5883510
417,"Vifon","Tu Quy Spicy Beef","Pack","Vietnam",3.5,1374037567
416,"Sapporo Ichiban","Chicken Flavor","Cup","USA",3.75,224216717
415,"Ve Wong","Artificial Classic Beef","Bowl","Taiwan",3.75,876399246
414,"Doll","Artificial Beef Flavor","Pack","Hong Kong",2.5,919409968
413,"Souper","Alimentary Paste Chow Mein","Pack","Taiwan",4.25,108789347
412,"Maruchan","Midori No Tanuki","Bowl","Japan",4.125,2117995635
411,"Sunlee","Cr

311,"Vifon","Tasty Mushroom","Pack","Vietnam",2,1752283078
310,"Lucky Me!","La Paz Batchoy Beef","Pack","Philippines",2.5,236794548
309,"Paldo","Dosirac Beef","Tray","South Korea",3.75,1572260069
308,"Tradition","Imitation Chicken Vegetarian","Pack","USA",3.75,133956434
307,"Royal Umbrella","Vegetarian Tom Yum","Cup","Thailand",2.75,897839971
306,"Rhee Bros Assi","Rice Noodle Anchovy Flavor","Tray","South Korea",3.75,1347616138
305,"Lucky Me!","Supreme Special Beef Carne De Vaca","Bowl","Philippines",3.75,1617202168
304,"Mama","Pho Bo Rice Noodle Artificial Beef","Bowl","Thailand",3.5,254096565
303,"Paldo","Dosirac Artificial Chicken","Tray","South Korea",3.25,1628019785
302,"Samyang","Bowl Noodle Picante Beef","Bowl","South Korea",3.75,225031231
301,"Dragonfly","Guilin Style Rice Vermicelli","Bowl","China",4.5,203891133
300,"Wu Mu","Steam Spinach Ramen With Onion","Pack","Taiwan",5,653794257
299,"Lucky Me!","Supreme Sotanghon Artificial Chicken Vermicelli ","Bowl","Philippines",3.75,9

195,"Myojo","Chukazanmai Served Cool Sesame Flavored Rice Vinegar Sesame","Pack","Japan",3,624319548
194,"Saigon Ve Wong","Kung Fu Instant Rice Noodles Artificial Chicken","Pack","Vietnam",3.5,904381161
193,"Boss","Shrimp Tom Yum Flavor","Bowl","Thailand",2.5,1680917220
192,"Paldo","Soy Flavor Noodle","Pack","South Korea",4.25,601159275
191,"Wu Mu","Steamed Noodle Chinese Herbs","Pack","Taiwan",3.25,180889026
190,"Maggi","Perencah Tom Yam","Pack","Malaysia",2.8999999999999999,974402836
189,"Nissin","Cup Noodles Manchurian","Cup","India",3,1227163140
188,"Nongshim","Udon","Pack","South Korea",4,2138145899
187,"Paldo","Hwa Ramyun Soy Peptide","Pack","South Korea",4,657022054
186,"Paldo","Teumsae Ramyun","Pack","South Korea",3.5,962991071
185,"Shirakiku","Sanukiya Udon","Bowl","Japan",3.75,2049413348
184,"Myojo","Chukazanmai Soy Sauce","Pack","Japan",3.75,1342424128
183,"Nissin","Bowl Noodles Hot & Spicy Chicken","Bowl","USA",2,438150098
182,"Myojo","Chukazanmai Soy Sauce chili Oil","Pack

79,"Sapporo Ichiban","Kitsune Udon","Pack","Japan",3.25,367083544
78,"Nissin","Demae Ramen Seafood","Pack","Japan",2.5,750232
77,"Nissin","Demae Ramen Prawn","Pack","Japan",3.25,2119645023
76,"Indomie","Meatball Flavor","Pack","Indonesia",2.5,827477461
75,"Indomie","Mi Goreng Rendang","Pack","Indonesia",4.5,1550579437
74,"Nissin","Demae Ramen Shoyu","Pack","Japan",4,1569242560
73,"Nissin","Demae Ramen Five Spices Artificial Beef","Pack","Japan",3.5,534236961
72,"Nissin","Demae Ramen Tonkatsu Artificial Pork","Pack","Japan",3.5,201872574
71,"Nissin","Demae Ramen Satay","Pack","Japan",2.5,399196315
70,"Nissin","Demae Ramen XO Sauce Seafood","Pack","Japan",4,1381795694
69,"Baijia","Spicy Fei- Chang Sweet Potato Noodle","Pack","China",1.75,159915105
68,"Wei Lih","Jah Jan Mien Instant Noodle With Fried Soybean Paste","Pack","Taiwan",4,469762960
67,"Maruchan","Ramen Noodle Soup Lime Chili Shrimp","Pack","USA",2,41764287
66,"Maruchan","Ramen Noodle Soup Pork","Pack","USA",4,1702597256
65,"Nis

This outputs very large positive integer. Sometimes we need random numbers within a certain range. Let's say between 0 and 10. Using modulus operator, 

In [20]:
aq_pp -f,+1 $file -d $cols -eval i:row '$random%10'

"reviewID","brand","variety","style","country","stars","row"
2580,"New Touch","T's Restaurant Tantanmen ","Cup","Japan",3.75,3
2579,"Just Way","Noodles Spicy Hot Sesame Spicy Hot Sesame Guan-miao Noodles","Pack","Taiwan",1,7
2578,"Nissin","Cup Noodles Chicken Vegetable","Cup","USA",2.25,8
2577,"Wei Lih","GGE Ramen Snack Tomato Flavor","Pack","Taiwan",2.75,1
2576,"Ching's Secret","Singapore Curry","Pack","India",3.75,7
2575,"Samyang Foods","Kimchi song Song Ramen","Pack","South Korea",4.75,9
2574,"Acecook","Spice Deli Tantan Men With Cilantro","Cup","Japan",4,8
2573,"Ikeda Shoku","Nabeyaki Kitsune Udon","Tray","Japan",3.75,3
2572,"Ripe'n'Dry","Hokkaido Soy Sauce Ramen","Pack","Japan",0.25,1
2571,"KOKA","The Original Spicy Stir-Fried Noodles","Pack","Singapore",2.5,2
2570,"Tao Kae Noi","Creamy tom Yum Kung Flavour","Pack","Thailand",5,6
2569,"Yamachan","Yokohama Tonkotsu Shoyu","Pack","USA",5,7
2568,"Nongshim","Mr. Bibim Stir-Fried Kimchi Flavor","Pack","South Korea",4.25,9
2567,"Nissin"

2463,"Nissin","Gekikara Ramen Rasa Pedas","Pack","Indonesia",3.25,5
2462,"TTL","Sesame Oil Chicken Noodle With Rice Wine","Bowl","Taiwan",5,8
2461,"Fujiwara","Hokkaido Hakodate Shio Ramen","Pack","Japan",4.5,2
2460,"Daifuku","Katsuo Bowl Udon","Bowl","USA",5,0
2459,"Nissin","Cup Noodles MUG Shoyu & Seafood","Pack","Japan",3.5,4
2457,"KOKA","Instant Noodles Chicken Satay Flavour","Pack","Singapore",3.5,9
2456,"KOKA","Signature Spicy Singapore Fried Noodles Instant Noodles","Pack","Singapore",4.25,3
2455,"KOKA","Creamy Soup With Crushed Noodles Sweet Corn Flavor","Cup","Singapore",5,1
2454,"KOKA","Silk Beef Pho Flavor Instant Rice Fettuccine","Bowl","Singapore",3.25,4
2453,"KOKA","Delight Spicy Sesame Flavor Instant Non-Fried Noodles","Pack","Singapore",4.5,5
2452,"KOKA","Signature Laksa Singapura Instant Noodles","Bowl","Singapore",5,6
2451,"KOKA","Delight Spicy Black Pepper Flavor Instant Non-Fried Noodles","Pack","Singapore",5,6
2450,"KOKA","Purple Wheat Noodles Chili & Lime Flavor","

2356,"Nongshim","Spicy Shrimp Cup Noodle","Cup","South Korea",4,6
2355,"Marutai","Nagasaki Agodashi Shoyu Ramen","Pack","Japan",4,7
2354,"Vifon","Viet Cuisine Bun Rieu Cua Sour Crab Soup Instant Rice Vermicelli","Bowl","Vietnam",5,2
2353,"Nissin","Cup Noodles Rich Garlic, Egg Yolk & Oxtail","Cup","Japan",4,8
2352,"Uni-President","Minced Pork Flavor Instant Noodles","Pack","Taiwan",1,1
2351,"1 To 3 Noodles","Chatpat Masala","Pack","India",4,8
2350,"Nissin","Cup Noodles BIG XO Sauce Seafood Flavour","Cup","Hong Kong",4,7
2349,"Fantastic","Noodles Chicken & Corn Flavour","Cup","Australia",3,6
2348,"Ripe'n'Dry","Hokkaido Salt Ramen","Pack","Japan",3,1
2347,"Nissin","Demae Iccho Tonkotsu Flavour Instant Noodle (Bowl Noodle)","Bowl","Hong Kong",3,3
2346,"Knorr","Chatt Patta Instant Noodles","Pack","Pakistan",1,6
2345,"Sempio","Clam Flavor Soup Kal-guksu","Pack","South Korea",5,1
2344,"Nissin","Cup Noodles Sopa Nissin Sabor A Pollo","Cup","Mexico",4,5
2343,"Nissin","Maxi Sopa Nissin Cuchareab

2249,"Love Cook","Sun Dried Noodle - Fruity Soy Bean Paste","Pack","Taiwan",4,4
2248,"Nongshim","Shin Noodle Soup","Cup","USA",5,3
2247,"Nissin","Sapporo Noukou Miso Ramen","Bowl","Japan",5,2
2246,"Dream Kitchen","Soy Sauce","Cup","USA",4,7
2245,"Nissin","Vietnamese Chicken Coriander Flavour Vermicelli","Cup","Hong Kong",4,5
2244,"Kang Shi Fu","Artificial Spicy Beef Flavour","Pack","Hong Kong",4,4
2243,"Nissin","Cup Noodles Black Pepper Crab Flavour","Cup","Hong Kong",4,9
2242,"Great Value","Beef Style Noodles With Vegetables","Cup","Canada",3,8
2241,"Fuku","Spicy Soup Instant Noodle Beef Flavour","Pack","Hong Kong",4,8
2240,"Wang","Rice Noodle Anchovy Flavor","Bowl","South Korea",3,2
2239,"Nissin","Cup Noodles Chicken Flavor Ramen Noodle Soup (New Recipe)","Cup","USA",4,2
2238,"Asian Thai Foods","Rumpum Gundruk Flavour","Pack","Nepal",1,8
2237,"Nissin","Cup Noodle Light+ Bagna Cauda","Cup","Japan",5,5
2236,"Maruchan","Gotsumori Koku Tonkotsu Ramen","Bowl","Japan",4,0
2235,"Fashion Foo

2139,"Fuku","Superior Soup Instant Noodles (Mfg in Thailand)","Pack","Hong Kong",4,5
2138,"Fuku","Superior Soup Instant Noodles (Mfg in Vietnam)","Pack","Hong Kong",4,1
2137,"Nissin","Demae Ramen Spicy Curry Flavour Instant Noodles","Pack","Hong Kong",4,7
2136,"Nissin","Demae Ramen Roast Beef Flavour Instant Noodles","Pack","Hong Kong",4,9
2135,"Doll","Instant Noodle Supreme Abalone And Chicken Flavour","Pack","Hong Kong",2,1
2134,"Four Seas","Chicken Flavour Instant Noodle","Pack","Hong Kong",4,5
2133,"Nissin","Kimchi Chikin Ramen Donburi","Bowl","Japan",5,0
2132,"Maruchan","Instant Lunch Chipotle Chicken Flavor Ramen Noodle Soup","Cup","USA",5,8
2131,"Myojo","Mee Goreng Thai Tom Yum Flavour","Pack","Singapore",4,9
2130,"Kang Shi Fu","Artificial Salted Beef With Mushrooms Flavour","Pack","Hong Kong",4,3
2129,"CarJEN","Hot & Spicy Dry Curry With Chives Soup","Pack","Malaysia",5,2
2128,"Sapporo Ichiban","Taimeiken Yousyoku Yasangatsukutta Omumen","Cup","Japan",4,0
2127,"Suimin","Noodles

...

533,"Paldo","Gomtang","Pack","South Korea",4.75,8
532,"Doll","Spicy Tomkotsu","Pack","Hong Kong",4.25,4
531,"Yum Yum","Beef","Pack","Thailand",3.25,4
530,"Mee Jang","Artificial Shrimp Tom yum","Bowl","Thailand",3.5,0
529,"Sapporo Ichiban","Hot Spicy Chicken","Pack","USA",2,2
528,"Unox","Good Noodles Vegetable","Pack","Holland",3.75,4
527,"Little Cook","Wheat Gluten Stewed Duck Flavoured","Bowl","Thailand",2.75,8
526,"Annie Chun's","Soy Ginger Ramen","Pack","USA",1.5,7
525,"JML","Artificial Spicy Hot Beef","Pack","China",3.75,9
524,"Nissin","Cup Noodles Spicy Seafood","Cup","Hong Kong",3.75,0
523,"Dragonfly","Artificial Hot & Sour Shrimp","Bowl","China",4.5,2
522,"Samyang","Hot Beef","Pack","South Korea",4.25,2
521,"Menraku","Soy Sauce","Bowl","Japan",4.25,4
520,"Nan Hsing","Vegetarian Rice Noodles","Bowl","Taiwan",0.25,7
519,"Snapdragon","Beijing Vegetable Rice Noodle","Cup","Singapore",2,7
518,"Dragonfly","Artificial Chicken","Bowl","China",4.5,8
517,"Thai Kitchen","Thai Ginger Rice N

397,"Quickchow","Quickie Mami La Paz Batchoy","Cup","Philippines",3,3
396,"Vina Acecook","Bestcook Hot spicy Tom Yum Shrimp","Bowl","Vietnam",4.25,3
395,"Ottogi","Jin Ramen (Mild)","Cup","South Korea",3,9
394,"Paldo","Hwa Cup Hot Spicy","Cup","South Korea",3.5,6
393,"Kamfen","Dried Mix Noodles XO Sauce","Tray","China",2,2
392,"Nissin","Yakisoba Noodles Karashi","Tray","Japan",5,8
391,"Myojo","Hyoubanya No Chukasoba Oriental","Pack","Japan",4.25,3
390,"Menraku","Curry Soba","Bowl","Japan",4,3
389,"Mama","Vegetarian Tom Yum","Pack","Thailand",2.75,5
388,"Maruchan","Instant Lunch Cheddar Cheese","Cup","USA",2.75,6
387,"Nissin","Bowl Noodles Rich & Savory Beef","Bowl","USA",3.75,7
386,"US Canning","Fu Chang Chinese Noodle Company Pork Seafood","Tray","USA",0,9
385,"Ve Wong","Hot Of Hots Artificial Black Peppered Beef","Bowl","Taiwan",1.25,5
384,"Nissin","Demae Iccho Tom Yam Goong Macaroni","Pack","Japan",4,8
383,"Lucky Me!","Chicken Na Chicken","Pack","Philippines",2.75,3
382,"Lucky Me!","

270,"Wu Mu","Dried Noodle With BBQ Pork","Pack","Taiwan",2,7
269,"Paldo","Spicy Shrimp Bowl","Bowl","South Korea",3.5,0
268,"Nissin","Cup Noodles Spicy Chile Chicken","Cup","USA",3.5,9
267,"Mama","Pork","Cup","Thailand",2,3
266,"Vina Acecook","Mi Lau Thai Shrimp","Pack","Vietnam",4,4
265,"Nissin","Cup Noodles Chicken","Cup","USA",3.5,5
264,"Vina Acecook","Hao Hao Sour Hot Shrimp","Pack","Vietnam",3.75,8
263,"Nissin","Chow Mein Kung Pao Chicken","Tray","USA",1.5,4
262,"Vina Acecook","Daily Beef Ball","Pack","Vietnam",3.75,8
261,"Vina Acecook","Oh! Ricey Chicken","Pack","Vietnam",3.75,8
260,"Nissin","Bowl Noodles Hot & Spicy Shrimp","Bowl","USA",3.5,5
259,"Vina Acecook","Mi Lau Thai Seafood","Pack","Vietnam",4,3
258,"Royal Umbrella","Tom Yum Shrimp","Cup","Thailand",3.25,6
257,"Nissin","Cup Noodles Beef","Cup","USA",3.5,8
256,"Nissin","Cup Noodles Shrimp","Cup","USA",3.5,6
255,"Mama","Oriental Kitchen Spicy Seafood","Cup","Thailand",3.75,7
254,"Vina Acecook","Daily Hot & Spicy Tom Yun","

135,"Unif / Tung-I","Fresh Shrimp","Pack","Taiwan",3.25,4
134,"Global Inspiration","Gyeong-Gi-Do Rice","Bowl","South Korea",2.25,8
133,"Baijia","Hot & Sour Rice Noodle","Pack","China",0,7
132,"Ve Wong","Artificial Peppered Beef","Pack","Taiwan",3.5,0
131,"Unif","100 Artificial Shallot Chicken","Pack","Taiwan",3.25,4
130,"Sapporo Ichiban","Miso Soybean Paste","Pack","Japan",3.5,8
129,"Nongshim","Mupama Tang Myun","Pack","South Korea",3.5,0
128,"Mama","Artificial PA-LO Duck","Pack","Thailand",0.10000000000000001,0
127,"Vifon","Mi Chay Vegetarian","Pack","Vietnam",2.7999999999999998,2
126,"Mama","Tom Yam Koong Rice Vermicelli","Pack","Thailand",1.5,6
125,"Shirakiku","Japanese Style Tokusen Karami","Pack","Japan",3,6
124,"Nissin","Yakisoba With Mayonnaise/Mustard Packet","Tray","Japan",5,8
123,"Wai Wai","Minced Pork Soeng Kreung","Pack","Thailand",3.7000000000000002,8
122,"Doll","Shrimp Wonton","Pack","Hong Kong",4,4
121,"Ottogi","Bekse Curry","Pack","South Korea",4,3
120,"Samyang","Beef",

4,"Wai Wai","Oriental Style Instant Noodles","Pack","Thailand",1,5
3,"Wai Wai","Tom Yum Shrimp","Pack","Thailand",2,2
2,"Wai Wai","Tom Yum Chili Flavor","Pack","Thailand",2,6
1,"Westbrae","Miso Ramen","Pack","USA",0.5,5


Let's take a look at bitwise operator, which performs bitwise logical operation on decimal numbers.

For this, we'll use different file containing binary numbers, which looks like this

number|mask
---|---
1|981
290|90
31|12
79|56
10|874

Let's perform `|`(bitwise OR) operator on `numbers` column, with a constant 32. The result will be stored in the new column `i:result`.

In [34]:
aq_pp -f,+1 data/aq_pp/bitwise.csv -d i:number i:mask -eval i:result 'number | 32'

"number","mask","result"
1,981,33
290,90,290
31,12,63
79,56,111
10,874,42
